In [69]:
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=be36302d7007d1e45405b2fb93e7dc6c9ae6fe45577276b10b57d7a41c7f9b60
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [70]:
import tensorflow as tf


In [71]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

CPU RAM Free: 8.3 GB
GPU 0 ... Mem Free: 14664MB / 15079MB | Utilization   3%


In [72]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 600
from tqdm import tqdm
import gc

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.decomposition import PCA

from keras.layers.normalization import BatchNormalization

from keras.models import Sequential, Model

from keras.layers import Input, Embedding, Dense, Activation, Dropout, Flatten

from keras import regularizers 

import keras

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GroupKFold

In [73]:
def init():
    np.random.seed = 0
    
init()

In [74]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

def smape2D(y_true, y_pred):
    return smape(np.ravel(y_true), np.ravel(y_pred))
    
def smape_mask(y_true, y_pred, threshold):
    denominator = (np.abs(y_true) + np.abs(y_pred)) 
    diff = np.abs(y_true - y_pred) 
    diff[denominator == 0] = 0.0
    
    return diff <= (threshold / 2.0) * denominator

In [75]:
path='/content/drive/My Drive/GoogleColab/Web_Traffic_Dataset/'
max_size = 181 # number of days in 2015 with 3 days before end

offset = 1/2

train_all = pd.read_csv(path+"train_2.csv")
#test_all_id = pd.read_csv(path+'key_2.csv')
train_all.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,...,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-

In [76]:
test_all_id.head()

,Page,Id,Date
0,007_スペクター_ja.wikipedia.org_all-access_all-agents,0b293039387a,2017-09-13
1,007_スペクター_ja.wikipedia.org_all-access_all-agents,7114389dd824,2017-09-14
2,007_スペクター_ja.wikipedia.org_all-access_all-agents,057b02ff1f09,2017-09-15
3,007_スペクター_ja.wikipedia.org_all-access_all-agents,bd2aca21caa3,2017-09-16
4,007_スペクター_ja.wikipedia.org_all-access_all-agents,c0effb42cdd5,2017-09-17


In [77]:
# Reduction of dataset (just take 10% of dataset)
from sklearn.model_selection import train_test_split #library

def sampling_dataset(dataset):
# dataset: pandas dataframe for the population
# data_keep: sampled dataframe based on train_test_split (we will keep this for further processing)
# data_discard: sampled datafram that we remove (we will remove this dataset)
# train_size=0.1  : samples 10% of dataset from the popolation and store at variable data_keep
# random_state=42 : to reproduce same result

  data_keep, data_discard=train_test_split(dataset,train_size=0.05, random_state=42)
  print('Total size and sampled size')
  print(dataset.shape,data_keep.shape)
  data_keep.sort_index(inplace=True)
  data_keep.reset_index(drop=True, inplace=True)
  return data_keep

#eg-
train_all=sampling_dataset(train_all)
# test_all_id=sampling_dataset(test_all_id)

Total size and sampled size
(145063, 804) (7253, 804)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [78]:
train_all.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,...,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-

In [79]:
all_page = train_all.Page.copy()
train_key = train_all[['Page']].copy()
train_all = train_all.iloc[:,1:] * offset 
train_all.head()

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,2016-04-25,...,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,20

In [80]:
def get_date_index(date, train_all=train_all):
    for idx, c in enumerate(train_all.columns):
        if date == c:
            break
    if idx == len(train_all.columns):
        return None
    return idx

In [81]:
get_date_index('2016-09-13')

440

In [82]:
get_date_index('2016-09-10')

437

In [83]:
train_all.shape[1] - get_date_index('2016-09-10')

366

In [84]:
get_date_index('2017-09-10') - get_date_index('2016-09-10')

365

In [85]:
trains = []
tests = []
train_end = get_date_index('2016-09-10') + 1
test_start = get_date_index('2016-09-13')

for i in range(-3,4):
    train = train_all.iloc[ : , (train_end - max_size + i) : train_end + i].copy().astype('float32')
    test = train_all.iloc[:, test_start + i : (63 + test_start) + i].copy().astype('float32')
    train = train.iloc[:,::-1].copy().astype('float32')
    trains.append(train)
    tests.append(test)

train_all = train_all.iloc[:,-(max_size):].astype('float32')
train_all = train_all.iloc[:,::-1].copy().astype('float32')

test_3_date = tests[3].columns

In [86]:
train_all.head()

,2017-09-10,2017-09-09,2017-09-08,2017-09-07,2017-09-06,2017-09-05,2017-09-04,2017-09-03,2017-09-02,2017-09-01,2017-08-31,2017-08-30,2017-08-29,2017-08-28,2017-08-27,2017-08-26,2017-08-25,2017-08-24,2017-08-23,2017-08-22,2017-08-21,2017-08-20,2017-08-19,2017-08-18,2017-08-17,2017-08-16,2017-08-15,2017-08-14,2017-08-13,2017-08-12,2017-08-11,2017-08-10,2017-08-09,2017-08-08,2017-08-07,2017-08-06,2017-08-05,2017-08-04,2017-08-03,2017-08-02,2017-08-01,2017-07-31,2017-07-30,2017-07-29,2017-07-28,2017-07-27,2017-07-26,2017-07-25,2017-07-24,2017-07-23,2017-07-22,2017-07-21,2017-07-20,2017-07-19,2017-07-18,2017-07-17,2017-07-16,2017-07-15,2017-07-14,2017-07-13,2017-07-12,2017-07-11,2017-07-10,2017-07-09,2017-07-08,2017-07-07,2017-07-06,2017-07-05,2017-07-04,2017-07-03,2017-07-02,2017-07-01,2017-06-30,2017-06-29,2017-06-28,2017-06-27,2017-06-26,2017-06-25,2017-06-24,2017-06-23,2017-06-22,2017-06-21,2017-06-20,2017-06-19,2017-06-18,2017-06-17,2017-06-16,2017-06-15,2017-06-14,2017-06-13,2017-06-12,2017-06-11,2017-06-10,2017-06-09,2017-06-08,2017-06-07,2017-06-06,2017-06-05,2017-06-04,2017-06-03,2017-06-02,2017-06-01,2017-05-31,2017-05-30,2017-05-29,2017-05-28,2017-05-27,2017-05-26,2017-05-25,2017-05-24,2017-05-23,2017-05-22,2017-05-21,2017-05-20,2017-05-19,2017-05-18,2017-05-17,2017-05-16,2017-05-15,2017-05-14,2017-05-13,2017-05-12,2017-05-11,2017-05-10,2017-05-09,2017-05-08,2017-05-07,2017-05-06,2017-05-05,2017-05-04,2017-05-03,2017-05-02,2017-05-01,2017-04-30,2017-04-29,2017-04-28,2017-04-27,2017-04-26,2017-04-25,2017-04-24,2017-04-23,2017-04-22,2017-04-21,2017-04-20,2017-04-19,2017-04-18,2017-04-17,2017-04-16,2017-04-15,2017-04-14,2017-04-13,2017-04-12,2017-04-11,2017-04-10,2017-04-09,2017-04-08,2017-04-07,2017-04-06,2017-04-05,2017-04-04,2017-04-03,2017-04-02,2017-04-01,2017-03-31,2017-03-30,2017-03-29,2017-03-28,2017-03-27,2017-03-26,2017-03-25,2017-03-24,2017-03-23,2017-03-22,2017-03-21,2017-03-20,2017-03-19,2017-03-18,2017-03-17,2017-03-16,2017-03-15,2017-03-14
0,9.0,21.5,14.5,33.5,24.0,17.0,15.5,12.5,16.5,15.5,10.0,9.0,12.0,19.0,17.0,16.0,16.0,47.0,24.5,20.5,14.0,10.5,19.0,24.0,15.5,15.0,13.0,18.0,31.0,21.5,19.5,9.5,17.0,34.5,83.0,50.5,83.0,51.5,10.5,19.0,45.0,56.5,12.0,8.5,10.5,14.0,70.5,17.5,25.5,8.5,6.0,7.5,15.5,16.5,26.0,51.5,10.5,11.5,9.5,11.5,11.0,8.5,14.0,7.5,18.5,20.5,26.0,12.0,14.0,12.5,24.0,19.5,29.5,13.0,11.5,7.5,7.0,13.0,12.0,9.5,12.0,25.0,11.5,10.5,27.0,15.5,12.5,19.5,14.0,18.0,16.5,9.0,14.0,13.5,36.0,11.5,17.0,10.0,16.5,16.5,19.0,14.5,12.5,12.5,20.5,7.5,11.0,11.0,8.0,9.0,11.5,9.0,12.0,16.0,19.0,12.5,15.0,25.5,22.5,10.5,9.0,5.0,10.0,12.5,8.0,7.0,11.5,13.5,15.0,18.5,8.5,12.0,12.5,9.0,17.5,15.5,5.5,14.0,20.0,9.5,10.0,17.5,12.0,17.5,10.0,6.0,13.5,21.5,8.5,9.0,8.5,12.0,13.5,12.0,16.0,15.0,9.0,13.0,31.5,15.0,12.0,18.0,14.0,15.0,19.0,10.0,8.0,4.5,9.5,16.5,12.5,8.5,11.5,16.0,15.5,22.5,35.0,15.0,15.5,20.5,24.0
1,NaN,NaN,0.0,0.5,1.5,NaN,0.0,NaN,NaN,NaN,0.5,NaN,1.0,2.5,3.5,1.5,2.0,3.0,10.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.5,0.0,NaN,NaN,0.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.5,1.5,1.0,NaN,NaN,NaN,NaN,0.5,0.5,0.0,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,0.5,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,0.5,NaN,0.5,NaN,NaN,0.5,NaN,NaN,0.5,0.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN
2,1.0,7.0,3.5,7.5,11.5,6.0,11.0,7.5,6.0,8.5,15.5,7.5,8.0,12.5,10.0,11.0,23.5,17.0,17.5,3.0,3.5,6.0,3.0,6.5,4.0,5.0,3.5,6.5,16.0,4.5,5.5,3.0,4.5,7.5,11.0,14.0,14.5,4.5,6.0,20.0,5.0,11.0,7.0,18.0,5.5,5.5,8.0,15.5,3.5,11.0,2.5,6.5,16.0,21.5,11.0,11.5,2.0,6.0,4.0,5.5,7.5,3.5,6.5,12.5,12.0,17.5,5.5,4.5,2.5,4.0,8.5,3.5,5.0,4.5,6.0,8.5,13.0,5.0,3.5,4.0,11.5,19.0,5.0,5.5,2.5,7.0,6.5,2.5,3.5,10.0,1.5,7.0,16.0,2.5,7

In [87]:
trains[3].head()

,2016-09-10,2016-09-09,2016-09-08,2016-09-07,2016-09-06,2016-09-05,2016-09-04,2016-09-03,2016-09-02,2016-09-01,2016-08-31,2016-08-30,2016-08-29,2016-08-28,2016-08-27,2016-08-26,2016-08-25,2016-08-24,2016-08-23,2016-08-22,2016-08-21,2016-08-20,2016-08-19,2016-08-18,2016-08-17,2016-08-16,2016-08-15,2016-08-14,2016-08-13,2016-08-12,2016-08-11,2016-08-10,2016-08-09,2016-08-08,2016-08-07,2016-08-06,2016-08-05,2016-08-04,2016-08-03,2016-08-02,2016-08-01,2016-07-31,2016-07-30,2016-07-29,2016-07-28,2016-07-27,2016-07-26,2016-07-25,2016-07-24,2016-07-23,2016-07-22,2016-07-21,2016-07-20,2016-07-19,2016-07-18,2016-07-17,2016-07-16,2016-07-15,2016-07-14,2016-07-13,2016-07-12,2016-07-11,2016-07-10,2016-07-09,2016-07-08,2016-07-07,2016-07-06,2016-07-05,2016-07-04,2016-07-03,2016-07-02,2016-07-01,2016-06-30,2016-06-29,2016-06-28,2016-06-27,2016-06-26,2016-06-25,2016-06-24,2016-06-23,2016-06-22,2016-06-21,2016-06-20,2016-06-19,2016-06-18,2016-06-17,2016-06-16,2016-06-15,2016-06-14,2016-06-13,2016-06-12,2016-06-11,2016-06-10,2016-06-09,2016-06-08,2016-06-07,2016-06-06,2016-06-05,2016-06-04,2016-06-03,2016-06-02,2016-06-01,2016-05-31,2016-05-30,2016-05-29,2016-05-28,2016-05-27,2016-05-26,2016-05-25,2016-05-24,2016-05-23,2016-05-22,2016-05-21,2016-05-20,2016-05-19,2016-05-18,2016-05-17,2016-05-16,2016-05-15,2016-05-14,2016-05-13,2016-05-12,2016-05-11,2016-05-10,2016-05-09,2016-05-08,2016-05-07,2016-05-06,2016-05-05,2016-05-04,2016-05-03,2016-05-02,2016-05-01,2016-04-30,2016-04-29,2016-04-28,2016-04-27,2016-04-26,2016-04-25,2016-04-24,2016-04-23,2016-04-22,2016-04-21,2016-04-20,2016-04-19,2016-04-18,2016-04-17,2016-04-16,2016-04-15,2016-04-14,2016-04-13,2016-04-12,2016-04-11,2016-04-10,2016-04-09,2016-04-08,2016-04-07,2016-04-06,2016-04-05,2016-04-04,2016-04-03,2016-04-02,2016-04-01,2016-03-31,2016-03-30,2016-03-29,2016-03-28,2016-03-27,2016-03-26,2016-03-25,2016-03-24,2016-03-23,2016-03-22,2016-03-21,2016-03-20,2016-03-19,2016-03-18,2016-03-17,2016-03-16,2016-03-15,2016-03-14
0,7.0,8.0,10.0,7.5,14.0,12.5,13.5,13.0,10.5,12.5,11.5,14.5,10.0,14.5,10.0,13.0,15.0,22.5,19.0,5.5,7.5,12.5,11.0,8.5,6.0,11.0,12.0,13.0,18.5,15.0,15.5,16.0,11.5,14.0,19.0,11.5,7.5,37.5,10.0,11.0,16.0,13.0,10.5,10.0,9.0,15.0,16.0,8.0,15.5,16.5,15.5,14.5,60.0,14.0,14.0,12.5,9.0,10.0,18.5,9.5,18.0,12.5,14.5,13.0,15.0,12.5,12.0,12.0,35.5,12.5,9.0,11.5,15.0,12.5,10.5,10.0,10.0,11.0,14.0,10.0,12.0,16.0,10.5,9.0,13.0,17.0,19.0,14.0,11.5,17.5,11.5,9.5,10.5,9.5,6.5,8.0,10.0,7.0,11.0,8.5,9.5,12.0,8.0,12.5,10.5,9.0,8.0,10.0,6.5,6.0,10.0,20.5,9.5,13.0,8.0,5.0,10.0,8.0,8.5,8.5,27.0,11.0,14.5,11.5,8.5,10.5,8.0,5.0,11.0,6.5,10.5,11.5,11.0,13.0,7.0,9.0,7.5,8.5,17.5,8.5,19.0,13.5,4.5,4.5,5.5,10.5,14.0,15.0,19.0,16.0,16.0,18.5,23.0,20.0,19.5,22.0,20.5,19.0,19.0,16.5,16.0,4.0,14.5,6.5,10.0,8.5,8.0,13.5,5.0,12.0,6.5,7.5,5.0,84.0,12.0,7.5,7.5,4.0,6.5,8.0,18.5
1,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.5,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,NaN,0.5,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.5
2,3.5,2.5,0.5,12.5,26.5,7.5,8.0,3.0,3.5,54.0,66.5,4.0,14.0,7.0,13.5,1.5,2.0,3.5,2.0,1.0,8.5,3.0,21.5,7.5,10.0,1.0,8.5,20.5,5.0,2.0,4.0,9.0,2.5,6.0,2.0,1.0,6.0,3.5,3.5,0.5,2.0,4.5,26.0,33.5,3.5,11.5,4.0,3.0,11.5,5.5,3.0,9.0,3.5,2.0,5.0,3.0,12.0,27.0,2.0,8.0,3.5,8.5,4.0,18.5,0.5,17.5,12.0,8.0,7.5,3.0,2.5,4.0,2.5,4.5,10.0,3.5,3.0,3.5,4.0,7.5,10.5,2.5,5.5,5.5,2.0,9.0,12.5,18.0,9.0,3.0,11.0,21.5,9.0,21.5,2.0,7.0,3.0,24.5,2.5,14.5,4.0

In [88]:
trains[0].head()

,2016-09-07,2016-09-06,2016-09-05,2016-09-04,2016-09-03,2016-09-02,2016-09-01,2016-08-31,2016-08-30,2016-08-29,2016-08-28,2016-08-27,2016-08-26,2016-08-25,2016-08-24,2016-08-23,2016-08-22,2016-08-21,2016-08-20,2016-08-19,2016-08-18,2016-08-17,2016-08-16,2016-08-15,2016-08-14,2016-08-13,2016-08-12,2016-08-11,2016-08-10,2016-08-09,2016-08-08,2016-08-07,2016-08-06,2016-08-05,2016-08-04,2016-08-03,2016-08-02,2016-08-01,2016-07-31,2016-07-30,2016-07-29,2016-07-28,2016-07-27,2016-07-26,2016-07-25,2016-07-24,2016-07-23,2016-07-22,2016-07-21,2016-07-20,2016-07-19,2016-07-18,2016-07-17,2016-07-16,2016-07-15,2016-07-14,2016-07-13,2016-07-12,2016-07-11,2016-07-10,2016-07-09,2016-07-08,2016-07-07,2016-07-06,2016-07-05,2016-07-04,2016-07-03,2016-07-02,2016-07-01,2016-06-30,2016-06-29,2016-06-28,2016-06-27,2016-06-26,2016-06-25,2016-06-24,2016-06-23,2016-06-22,2016-06-21,2016-06-20,2016-06-19,2016-06-18,2016-06-17,2016-06-16,2016-06-15,2016-06-14,2016-06-13,2016-06-12,2016-06-11,2016-06-10,2016-06-09,2016-06-08,2016-06-07,2016-06-06,2016-06-05,2016-06-04,2016-06-03,2016-06-02,2016-06-01,2016-05-31,2016-05-30,2016-05-29,2016-05-28,2016-05-27,2016-05-26,2016-05-25,2016-05-24,2016-05-23,2016-05-22,2016-05-21,2016-05-20,2016-05-19,2016-05-18,2016-05-17,2016-05-16,2016-05-15,2016-05-14,2016-05-13,2016-05-12,2016-05-11,2016-05-10,2016-05-09,2016-05-08,2016-05-07,2016-05-06,2016-05-05,2016-05-04,2016-05-03,2016-05-02,2016-05-01,2016-04-30,2016-04-29,2016-04-28,2016-04-27,2016-04-26,2016-04-25,2016-04-24,2016-04-23,2016-04-22,2016-04-21,2016-04-20,2016-04-19,2016-04-18,2016-04-17,2016-04-16,2016-04-15,2016-04-14,2016-04-13,2016-04-12,2016-04-11,2016-04-10,2016-04-09,2016-04-08,2016-04-07,2016-04-06,2016-04-05,2016-04-04,2016-04-03,2016-04-02,2016-04-01,2016-03-31,2016-03-30,2016-03-29,2016-03-28,2016-03-27,2016-03-26,2016-03-25,2016-03-24,2016-03-23,2016-03-22,2016-03-21,2016-03-20,2016-03-19,2016-03-18,2016-03-17,2016-03-16,2016-03-15,2016-03-14,2016-03-13,2016-03-12,2016-03-11
0,7.5,14.0,12.5,13.5,13.0,10.5,12.5,11.5,14.5,10.0,14.5,10.0,13.0,15.0,22.5,19.0,5.5,7.5,12.5,11.0,8.5,6.0,11.0,12.0,13.0,18.5,15.0,15.5,16.0,11.5,14.0,19.0,11.5,7.5,37.5,10.0,11.0,16.0,13.0,10.5,10.0,9.0,15.0,16.0,8.0,15.5,16.5,15.5,14.5,60.0,14.0,14.0,12.5,9.0,10.0,18.5,9.5,18.0,12.5,14.5,13.0,15.0,12.5,12.0,12.0,35.5,12.5,9.0,11.5,15.0,12.5,10.5,10.0,10.0,11.0,14.0,10.0,12.0,16.0,10.5,9.0,13.0,17.0,19.0,14.0,11.5,17.5,11.5,9.5,10.5,9.5,6.5,8.0,10.0,7.0,11.0,8.5,9.5,12.0,8.0,12.5,10.5,9.0,8.0,10.0,6.5,6.0,10.0,20.5,9.5,13.0,8.0,5.0,10.0,8.0,8.5,8.5,27.0,11.0,14.5,11.5,8.5,10.5,8.0,5.0,11.0,6.5,10.5,11.5,11.0,13.0,7.0,9.0,7.5,8.5,17.5,8.5,19.0,13.5,4.5,4.5,5.5,10.5,14.0,15.0,19.0,16.0,16.0,18.5,23.0,20.0,19.5,22.0,20.5,19.0,19.0,16.5,16.0,4.0,14.5,6.5,10.0,8.5,8.0,13.5,5.0,12.0,6.5,7.5,5.0,84.0,12.0,7.5,7.5,4.0,6.5,8.0,18.5,6.5,11.0,4.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.5,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,NaN,0.5,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.5,0.0,0.0,NaN
2,12.5,26.5,7.5,8.0,3.0,3.5,54.0,66.5,4.0,14.0,7.0,13.5,1.5,2.0,3.5,2.0,1.0,8.5,3.0,21.5,7.5,10.0,1.0,8.5,20.5,5.0,2.0,4.0,9.0,2.5,6.0,2.0,1.0,6.0,3.5,3.5,0.5,2.0,4.5,26.0,33.5,3.5,11.5,4.0,3.0,11.5,5.5,3.0,9.0,3.5,2.0,5.0,3.0,12.0,27.0,2.0,8.0,3.5,8.5,4.0,18.5,0.5,17.5,12.0,8.0,7.5,3.0,2.5,4.0,2.5,4.5,10.0,3.5,3.0,3.5,4.0,7.5,10.5,2.5,5.5,5.5,2.0,9.0,12.5,18.0,9.0,3.0,11.0,21.5,9.0,21.5,2.0,7.0,3.0,24.5,2.5,14.5,4.0,37.0,13.0,3

In [89]:
tests[3].head()

,2016-09-13,2016-09-14,2016-09-15,2016-09-16,2016-09-17,2016-09-18,2016-09-19,2016-09-20,2016-09-21,2016-09-22,2016-09-23,2016-09-24,2016-09-25,2016-09-26,2016-09-27,2016-09-28,2016-09-29,2016-09-30,2016-10-01,2016-10-02,2016-10-03,2016-10-04,2016-10-05,2016-10-06,2016-10-07,2016-10-08,2016-10-09,2016-10-10,2016-10-11,2016-10-12,2016-10-13,2016-10-14,2016-10-15,2016-10-16,2016-10-17,2016-10-18,2016-10-19,2016-10-20,2016-10-21,2016-10-22,2016-10-23,2016-10-24,2016-10-25,2016-10-26,2016-10-27,2016-10-28,2016-10-29,2016-10-30,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08,2016-11-09,2016-11-10,2016-11-11,2016-11-12,2016-11-13,2016-11-14
0,9.5,12.5,24.0,13.5,9.5,7.5,23.0,10.0,10.5,15.0,9.0,14.5,16.5,7.0,11.5,14.0,10.5,20.5,14.0,10.5,19.5,12.0,11.5,10.0,11.5,12.5,12.5,18.0,9.5,11.5,12.0,7.5,10.5,11.0,17.0,10.5,13.5,10.5,10.5,8.0,15.0,15.5,10.5,13.5,9.0,10.5,33.0,12.0,11.5,12.5,11.5,15.5,13.0,8.5,13.0,12.0,11.5,11.0,12.5,6.5,30.5,12.5,20.5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.5,2.0,20.5,36.5,4.5,1.0,9.5,2.5,30.0,2.5,4.0,2.0,8.5,2.0,1.5,1.5,0.5,4.5,70.0,52.0,52.0,20.0,51.5,25.0,18.0,35.0,22.0,11.5,8.5,10.0,25.5,28.0,38.0,10.5,13.0,7.0,5.0,6.5,3.0,4.0,24.0,29.5,11.5,7.5,6.0,6.0,9.0,12.5,22.0,12.0,14.5,2.5,14.5,3.0,4.5,9.5,5.0,5.0,17.5,0.5,3.5,3.0,46.5
3,3.0,11.5,15.0,21.5,19.5,4.5,28.0,6.0,3.5,11.0,4.0,5.0,22.5,21.5,10.0,4.5,6.0,23.0,23.5,14.5,5.5,16.5,6.0,9.5,7.5,10.5,33.0,11.0,8.5,6.5,31.5,15.5,42.0,13.0,22.5,8.5,12.0,11.5,148.5,34.0,12.0,5.0,7.0,18.5,9.5,16.0,37.0,14.0,10.5,11.0,17.0,7.0,14.0,17.0,7.5,8.0,20.5,7.5,5.0,7.0,34.0,8.0,3.0
4,12.0,7.0,7.0,8.0,10.0,6.0,8.0,7.0,6.0,7.5,9.0,6.5,8.0,9.5,15.5,13.0,9.0,8.0,4.5,25.0,12.0,5.0,8.0,9.5,13.0,5.5,37.0,20.0,7.5,7.5,9.5,10.5,6.0,10.0,9.5,5.0,4.0,9.5,6.0,6.5,9.0,3.5,10.0,12.5,7.5,6.5,7.0,18.0,9.5,11.0,10.5,7.0,5.0,7.0,8.0,12.0,6.0,4.0,8.0,3.5,186.0,7.0,8.5


In [90]:
data = [page.split('_') for page in tqdm(train_key.Page)]

access = ['_'.join(page[-2:]) for page in data]

site = [page[-3] for page in data]

page = ['_'.join(page[:-3]) for page in data]
page[:2]

train_key['PageTitle'] = page
train_key['Site'] = site
train_key['AccessAgent'] = access
train_key.head()

100%|██████████| 7253/7253 [00:00<00:00, 832593.11it/s]


,Page,PageTitle,Site,AccessAgent
0,Beyond_zh.wikipedia.org_all-access_spider,Beyond,zh.wikipedia.org,all-access_spider
1,File:Sebastiaosalgado2006.jpg_zh.wikipedia.org_all-access_spider,File:Sebastiaosalgado2006.jpg,zh.wikipedia.org,all-access_spider
2,GNZ48_zh.wikipedia.org_all-access_spider,GNZ48,zh.wikipedia.org,all-access_spider
3,Gu9udan_zh.wikipedia.org_all-access_spider,Gu9udan,zh.wikipedia.org,all-access_spider
4,IKON_zh.wikipedia.org_all-access_spider,IKON,zh.wikipedia.org,all-access_spider


# Implementation of log1p

In [91]:
train_norms = [np.log1p(train).astype('float32') for train in trains]
train_norms[3].head()

,2016-09-10,2016-09-09,2016-09-08,2016-09-07,2016-09-06,2016-09-05,2016-09-04,2016-09-03,2016-09-02,2016-09-01,2016-08-31,2016-08-30,2016-08-29,2016-08-28,2016-08-27,2016-08-26,2016-08-25,2016-08-24,2016-08-23,2016-08-22,2016-08-21,2016-08-20,2016-08-19,2016-08-18,2016-08-17,2016-08-16,2016-08-15,2016-08-14,2016-08-13,2016-08-12,2016-08-11,2016-08-10,2016-08-09,2016-08-08,2016-08-07,2016-08-06,2016-08-05,2016-08-04,2016-08-03,2016-08-02,2016-08-01,2016-07-31,2016-07-30,2016-07-29,2016-07-28,2016-07-27,2016-07-26,2016-07-25,2016-07-24,2016-07-23,2016-07-22,2016-07-21,2016-07-20,2016-07-19,2016-07-18,2016-07-17,2016-07-16,2016-07-15,2016-07-14,2016-07-13,2016-07-12,2016-07-11,2016-07-10,2016-07-09,2016-07-08,2016-07-07,2016-07-06,2016-07-05,2016-07-04,2016-07-03,2016-07-02,2016-07-01,2016-06-30,2016-06-29,2016-06-28,2016-06-27,2016-06-26,2016-06-25,2016-06-24,2016-06-23,2016-06-22,2016-06-21,2016-06-20,2016-06-19,2016-06-18,2016-06-17,2016-06-16,2016-06-15,2016-06-14,2016-06-13,2016-06-12,2016-06-11,2016-06-10,2016-06-09,2016-06-08,2016-06-07,2016-06-06,2016-06-05,2016-06-04,2016-06-03,2016-06-02,2016-06-01,2016-05-31,2016-05-30,2016-05-29,2016-05-28,2016-05-27,2016-05-26,2016-05-25,2016-05-24,2016-05-23,2016-05-22,2016-05-21,2016-05-20,2016-05-19,2016-05-18,2016-05-17,2016-05-16,2016-05-15,2016-05-14,2016-05-13,2016-05-12,2016-05-11,2016-05-10,2016-05-09,2016-05-08,2016-05-07,2016-05-06,2016-05-05,2016-05-04,2016-05-03,2016-05-02,2016-05-01,2016-04-30,2016-04-29,2016-04-28,2016-04-27,2016-04-26,2016-04-25,2016-04-24,2016-04-23,2016-04-22,2016-04-21,2016-04-20,2016-04-19,2016-04-18,2016-04-17,2016-04-16,2016-04-15,2016-04-14,2016-04-13,2016-04-12,2016-04-11,2016-04-10,2016-04-09,2016-04-08,2016-04-07,2016-04-06,2016-04-05,2016-04-04,2016-04-03,2016-04-02,2016-04-01,2016-03-31,2016-03-30,2016-03-29,2016-03-28,2016-03-27,2016-03-26,2016-03-25,2016-03-24,2016-03-23,2016-03-22,2016-03-21,2016-03-20,2016-03-19,2016-03-18,2016-03-17,2016-03-16,2016-03-15,2016-03-14
0,2.079442,2.197225,2.397895,2.140066,2.708050,2.602690,2.674149,2.639057,2.442347,2.602690,2.525729,2.740840,2.397895,2.740840,2.397895,2.639057,2.772589,3.157001,2.995732,1.871802,2.140066,2.602690,2.484907,2.251292,1.945910,2.484907,2.564949,2.639057,2.970414,2.772589,2.803360,2.833213,2.525729,2.708050,2.995732,2.525729,2.140066,3.650658,2.397895,2.484907,2.833213,2.639057,2.442347,2.397895,2.302585,2.772589,2.833213,2.197225,2.803360,2.862201,2.803360,2.740840,4.110874,2.708050,2.708050,2.602690,2.302585,2.397895,2.970414,2.351375,2.944439,2.602690,2.740840,2.639057,2.772589,2.602690,2.564949,2.564949,3.597312,2.602690,2.302585,2.525729,2.772589,2.602690,2.442347,2.397895,2.397895,2.484907,2.708050,2.397895,2.564949,2.833213,2.442347,2.302585,2.639057,2.890372,2.995732,2.708050,2.525729,2.917771,2.525729,2.351375,2.442347,2.351375,2.014903,2.197225,2.397895,2.079442,2.484907,2.251292,2.351375,2.564949,2.197225,2.602690,2.442347,2.302585,2.197225,2.397895,2.014903,1.945910,2.397895,3.068053,2.351375,2.639057,2.197225,1.791759,2.397895,2.197225,2.251292,2.251292,3.332205,2.484907,2.740840,2.525729,2.251292,2.442347,2.197225,1.791759,2.484907,2.014903,2.442347,2.525729,2.484907,2.639057,2.079442,2.302585,2.140066,2.251292,2.917771,2.251292,2.995732,2.674149,1.704748,1.704748,1.871802,2.442347,2.70805,2.772589,2.995732,2.833213,2.833213,2.970414,3.178054,3.044523,3.020425,3.135494,3.068053,2.995732,2.995732,2.862201,2.833213,1.609438,2.74084,2.014903,2.397895,2.251292,2.197225,2.674149,1.791759,2.564949,2.014903,2.140066,1.791759,4.442651,2.564949,2.140066,2.140066,1.609438,2.014903,2.197225,2.970414
1,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,0.405465,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,0.405465

In [92]:
train_all_norm = np.log1p(train_all).astype('float32')
train_all_norm.head()

,2017-09-10,2017-09-09,2017-09-08,2017-09-07,2017-09-06,2017-09-05,2017-09-04,2017-09-03,2017-09-02,2017-09-01,2017-08-31,2017-08-30,2017-08-29,2017-08-28,2017-08-27,2017-08-26,2017-08-25,2017-08-24,2017-08-23,2017-08-22,2017-08-21,2017-08-20,2017-08-19,2017-08-18,2017-08-17,2017-08-16,2017-08-15,2017-08-14,2017-08-13,2017-08-12,2017-08-11,2017-08-10,2017-08-09,2017-08-08,2017-08-07,2017-08-06,2017-08-05,2017-08-04,2017-08-03,2017-08-02,2017-08-01,2017-07-31,2017-07-30,2017-07-29,2017-07-28,2017-07-27,2017-07-26,2017-07-25,2017-07-24,2017-07-23,2017-07-22,2017-07-21,2017-07-20,2017-07-19,2017-07-18,2017-07-17,2017-07-16,2017-07-15,2017-07-14,2017-07-13,2017-07-12,2017-07-11,2017-07-10,2017-07-09,2017-07-08,2017-07-07,2017-07-06,2017-07-05,2017-07-04,2017-07-03,2017-07-02,2017-07-01,2017-06-30,2017-06-29,2017-06-28,2017-06-27,2017-06-26,2017-06-25,2017-06-24,2017-06-23,2017-06-22,2017-06-21,2017-06-20,2017-06-19,2017-06-18,2017-06-17,2017-06-16,2017-06-15,2017-06-14,2017-06-13,2017-06-12,2017-06-11,2017-06-10,2017-06-09,2017-06-08,2017-06-07,2017-06-06,2017-06-05,2017-06-04,2017-06-03,2017-06-02,2017-06-01,2017-05-31,2017-05-30,2017-05-29,2017-05-28,2017-05-27,2017-05-26,2017-05-25,2017-05-24,2017-05-23,2017-05-22,2017-05-21,2017-05-20,2017-05-19,2017-05-18,2017-05-17,2017-05-16,2017-05-15,2017-05-14,2017-05-13,2017-05-12,2017-05-11,2017-05-10,2017-05-09,2017-05-08,2017-05-07,2017-05-06,2017-05-05,2017-05-04,2017-05-03,2017-05-02,2017-05-01,2017-04-30,2017-04-29,2017-04-28,2017-04-27,2017-04-26,2017-04-25,2017-04-24,2017-04-23,2017-04-22,2017-04-21,2017-04-20,2017-04-19,2017-04-18,2017-04-17,2017-04-16,2017-04-15,2017-04-14,2017-04-13,2017-04-12,2017-04-11,2017-04-10,2017-04-09,2017-04-08,2017-04-07,2017-04-06,2017-04-05,2017-04-04,2017-04-03,2017-04-02,2017-04-01,2017-03-31,2017-03-30,2017-03-29,2017-03-28,2017-03-27,2017-03-26,2017-03-25,2017-03-24,2017-03-23,2017-03-22,2017-03-21,2017-03-20,2017-03-19,2017-03-18,2017-03-17,2017-03-16,2017-03-15,2017-03-14
0,2.302585,3.113515,2.740840,3.540959,3.218876,2.890372,2.803360,2.602690,2.862201,2.803360,2.397895,2.302585,2.564949,2.995732,2.890372,2.833213,2.833213,3.871201,3.238678,3.068053,2.708050,2.442347,2.995732,3.218876,2.803360,2.772589,2.639057,2.944439,3.465736,3.113515,3.020425,2.351375,2.890372,3.569533,4.430817,3.941582,4.430817,3.960813,2.442347,2.995732,3.828641,4.051785,2.564949,2.251292,2.442347,2.708050,4.269698,2.917771,3.277145,2.251292,1.945910,2.140066,2.803360,2.862201,3.295837,3.960813,2.442347,2.525729,2.351375,2.525729,2.484907,2.251292,2.708050,2.140066,2.970414,3.068053,3.295837,2.564949,2.708050,2.602690,3.218876,3.020425,3.417727,2.639057,2.525729,2.140066,2.079442,2.639057,2.564949,2.351375,2.564949,3.258096,2.525729,2.442347,3.332205,2.803360,2.602690,3.020425,2.708050,2.944439,2.862201,2.302585,2.708050,2.674149,3.610918,2.525729,2.890372,2.397895,2.862201,2.862201,2.995732,2.740840,2.602690,2.602690,3.068053,2.140066,2.484907,2.484907,2.197225,2.302585,2.525729,2.302585,2.564949,2.833213,2.995732,2.602690,2.772589,3.277145,3.157001,2.442347,2.302585,1.791759,2.397895,2.602690,2.197225,2.079442,2.525729,2.674149,2.772589,2.970414,2.251292,2.564949,2.602690,2.302585,2.917771,2.803360,1.871802,2.708050,3.044523,2.351375,2.397895,2.917771,2.564949,2.917771,2.397895,1.945910,2.674149,3.113515,2.251292,2.302585,2.251292,2.564949,2.674149,2.564949,2.833213,2.772589,2.302585,2.639057,3.481240,2.772589,2.564949,2.944439,2.708050,2.772589,2.995732,2.397895,2.197225,1.704748,2.351375,2.862201,2.602690,2.251292,2.525729,2.833213,2.803360,3.157001,3.583519,2.772589,2.803360,3.068053,3.218876
1,NaN,NaN,0.000000,0.405465,0.916291,NaN,0.000000,NaN,NaN,NaN,0.405465,NaN,0.693147,1.252763,1.504077,0.916291,1.098612,1.386294,2.397895,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.405465,0.000000,NaN,NaN,0.405465,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.405465,0.916291,0.693147,NaN,NaN,NaN,NaN,0.405465,0.405465,0.000000,0.000000,0.405465,NaN,

In [93]:
for i,test in enumerate(tests):
    first_day = i-2 # 2016-09-13 is a Tuesday
    test_columns_date = list(test.columns)
    test_columns_code = ['w%d_d%d' % (i // 7, (first_day + i) % 7) for i in range(63)]
    test.columns = test_columns_code

tests[3].head()


,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w0_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w1_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w2_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w3_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w4_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w5_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w6_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w7_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w8_d0
0,9.5,12.5,24.0,13.5,9.5,7.5,23.0,10.0,10.5,15.0,9.0,14.5,16.5,7.0,11.5,14.0,10.5,20.5,14.0,10.5,19.5,12.0,11.5,10.0,11.5,12.5,12.5,18.0,9.5,11.5,12.0,7.5,10.5,11.0,17.0,10.5,13.5,10.5,10.5,8.0,15.0,15.5,10.5,13.5,9.0,10.5,33.0,12.0,11.5,12.5,11.5,15.5,13.0,8.5,13.0,12.0,11.5,11.0,12.5,6.5,30.5,12.5,20.5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.5,2.0,20.5,36.5,4.5,1.0,9.5,2.5,30.0,2.5,4.0,2.0,8.5,2.0,1.5,1.5,0.5,4.5,70.0,52.0,52.0,20.0,51.5,25.0,18.0,35.0,22.0,11.5,8.5,10.0,25.5,28.0,38.0,10.5,13.0,7.0,5.0,6.5,3.0,4.0,24.0,29.5,11.5,7.5,6.0,6.0,9.0,12.5,22.0,12.0,14.5,2.5,14.5,3.0,4.5,9.5,5.0,5.0,17.5,0.5,3.5,3.0,46.5
3,3.0,11.5,15.0,21.5,19.5,4.5,28.0,6.0,3.5,11.0,4.0,5.0,22.5,21.5,10.0,4.5,6.0,23.0,23.5,14.5,5.5,16.5,6.0,9.5,7.5,10.5,33.0,11.0,8.5,6.5,31.5,15.5,42.0,13.0,22.5,8.5,12.0,11.5,148.5,34.0,12.0,5.0,7.0,18.5,9.5,16.0,37.0,14.0,10.5,11.0,17.0,7.0,14.0,17.0,7.5,8.0,20.5,7.5,5.0,7.0,34.0,8.0,3.0
4,12.0,7.0,7.0,8.0,10.0,6.0,8.0,7.0,6.0,7.5,9.0,6.5,8.0,9.5,15.5,13.0,9.0,8.0,4.5,25.0,12.0,5.0,8.0,9.5,13.0,5.5,37.0,20.0,7.5,7.5,9.5,10.5,6.0,10.0,9.5,5.0,4.0,9.5,6.0,6.5,9.0,3.5,10.0,12.5,7.5,6.5,7.0,18.0,9.5,11.0,10.5,7.0,5.0,7.0,8.0,12.0,6.0,4.0,8.0,3.5,186.0,7.0,8.5


In [94]:
tests[0].head()

,w0_d5,w0_d6,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w1_d5,w1_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w2_d5,w2_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w3_d5,w3_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w4_d5,w4_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w5_d5,w5_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w6_d5,w6_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w7_d5,w7_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w8_d5,w8_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4
0,7.0,9.5,10.5,9.5,12.5,24.0,13.5,9.5,7.5,23.0,10.0,10.5,15.0,9.0,14.5,16.5,7.0,11.5,14.0,10.5,20.5,14.0,10.5,19.5,12.0,11.5,10.0,11.5,12.5,12.5,18.0,9.5,11.5,12.0,7.5,10.5,11.0,17.0,10.5,13.5,10.5,10.5,8.0,15.0,15.5,10.5,13.5,9.0,10.5,33.0,12.0,11.5,12.5,11.5,15.5,13.0,8.5,13.0,12.0,11.5,11.0,12.5,6.5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN
2,3.5,3.0,25.0,10.5,2.0,20.5,36.5,4.5,1.0,9.5,2.5,30.0,2.5,4.0,2.0,8.5,2.0,1.5,1.5,0.5,4.5,70.0,52.0,52.0,20.0,51.5,25.0,18.0,35.0,22.0,11.5,8.5,10.0,25.5,28.0,38.0,10.5,13.0,7.0,5.0,6.5,3.0,4.0,24.0,29.5,11.5,7.5,6.0,6.0,9.0,12.5,22.0,12.0,14.5,2.5,14.5,3.0,4.5,9.5,5.0,5.0,17.5,0.5
3,6.0,22.5,16.0,3.0,11.5,15.0,21.5,19.5,4.5,28.0,6.0,3.5,11.0,4.0,5.0,22.5,21.5,10.0,4.5,6.0,23.0,23.5,14.5,5.5,16.5,6.0,9.5,7.5,10.5,33.0,11.0,8.5,6.5,31.5,15.5,42.0,13.0,22.5,8.5,12.0,11.5,148.5,34.0,12.0,5.0,7.0,18.5,9.5,16.0,37.0,14.0,10.5,11.0,17.0,7.0,14.0,17.0,7.5,8.0,20.5,7.5,5.0,7.0
4,12.0,30.5,6.0,12.0,7.0,7.0,8.0,10.0,6.0,8.0,7.0,6.0,7.5,9.0,6.5,8.0,9.5,15.5,13.0,9.0,8.0,4.5,25.0,12.0,5.0,8.0,9.5,13.0,5.5,37.0,20.0,7.5,7.5,9.5,10.5,6.0,10.0,9.5,5.0,4.0,9.5,6.0,6.5,9.0,3.5,10.0,12.5,7.5,6.5,7.0,18.0,9.5,11.0,10.5,7.0,5.0,7.0,8.0,12.0,6.0,4.0,8.0,3.5


In [95]:
for test in tests:
    test.fillna(0, inplace=True)

    test['Page'] = all_page
    test.sort_values(by='Page', inplace=True)
    test.reset_index(drop=True, inplace=True)

In [96]:
tests = [test.merge(train_key, how='left', on='Page', copy=False) for test in tests]

tests[3].head()


,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w0_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w1_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w2_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w3_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w4_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w5_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w6_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w7_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w8_d0,Page,PageTitle,Site,AccessAgent
0,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,7.0,13.5,17.0,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,2.0,0.0,15.0,1.5,0.0,2.0,2.0,1.0,1.5,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.5,0.5,3.0,0.5,0.5,0.5,1.0,4.0,1.0,0.5,1.5,0.5,0.0,1.0,1.5,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,1.0,1.0,2.5,0.0,1.0,1.0,0.5,0.5,1.5,1.0,0.5,0.5,2.0,2.0,0.0,2.0,0.5,0.5,0.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,92.5,90.5,98.5,82.5,71.0,79.5,128.0,87.5,92.5,82.0,111.5,74.0,59.5,89.5,133.5,111.5,85.0,108.5,71.5,154.0,137.0,135.0,104.5,94.0,66.5,64.5,82.5,146.0,93.0,109.5,125.5,85.5,70.5,80.0,90.0,89.5,103.5,98.5,82.0,65.0,85.5,140.5,105.0,92.5,86.0,77.0,61.5,59.5,73.5,75.0,81.5,108.5,125.5,105.5,135.0,145.0,116.0,101.0,130.0,122.0,112.5,140.5,131.5,(1)_Ceres_de.wikipedia.org_desktop_all-agents,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,26.0,27.5,15.5,12.0,20.0,20.5,15.5,38.0,15.5,27.5,21.5,38.5,22.0,39.0,16.0,15.0,28.5,35.0,22.5,24.0,18.0,27.0,24.0,27.5,23.0,19.0,22.0,28.0,15.0,23.5,37.0,23.0,26.5,28.5,23.5,20.5,23.0,20.5,23.0,25.0,18.5,18.0,15.0,25.0,20.5,13.0,18.5,20.5,20.0,26.5,26.0,19.5,20.5,27.5,26.5,21.0,21.0,13.0,23.5,17.5,24.0,18.0,17.0,0.999…_zh.wikipedia.org_all-access_all-agents,0.999…,zh.wikipedia.org,all-access_all-agents
4,52.0,44.0,60.0,104.5,35.0,36.5,105.5,85.0,116.0,52.0,54.5,24.0,50.0,36.0,22.5,30.0,30.0,27.5,24.5,13.0,51.0,12.5,24.5,27.5,12.5,16.0,13.5,19.0,19.0,20.5,28.5,13.5,12.5,10.0,12.5,12.0,17.0,14.5,12.5,15.5,15.5,18.0,25.0,41.0,21.5,22.5,13.0,17.5,27.0,25.0,20.0,23.0,17.5,21.0,13.0,23.5,11.5,16.5,15.5,19.5,12.5,21.0,16.0,1._September_de.wikipedia.org_desktop_all-agents,1._September,de.wikipedia.org,desktop_all-agents


In [97]:
tests[0].head()

,w0_d5,w0_d6,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w1_d5,w1_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w2_d5,w2_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w3_d5,w3_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w4_d5,w4_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w5_d5,w5_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w6_d5,w6_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w7_d5,w7_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w8_d5,w8_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,Page,PageTitle,Site,AccessAgent
0,3.5,19.0,5.5,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,0.0,0.0,0.0,2.0,0.0,15.0,1.5,0.0,2.0,2.0,1.0,1.5,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.5,0.5,3.0,0.5,0.5,0.5,1.0,4.0,1.0,0.5,1.5,0.5,0.0,1.0,1.5,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,1.0,1.0,2.5,0.0,1.0,1.0,0.5,0.5,1.5,1.0,0.5,0.5,2.0,2.0,0.0,2.0,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,81.5,158.0,117.5,92.5,90.5,98.5,82.5,71.0,79.5,128.0,87.5,92.5,82.0,111.5,74.0,59.5,89.5,133.5,111.5,85.0,108.5,71.5,154.0,137.0,135.0,104.5,94.0,66.5,64.5,82.5,146.0,93.0,109.5,125.5,85.5,70.5,80.0,90.0,89.5,103.5,98.5,82.0,65.0,85.5,140.5,105.0,92.5,86.0,77.0,61.5,59.5,73.5,75.0,81.5,108.5,125.5,105.5,135.0,145.0,116.0,101.0,130.0,122.0,(1)_Ceres_de.wikipedia.org_desktop_all-agents,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,25.0,18.0,21.0,26.0,27.5,15.5,12.0,20.0,20.5,15.5,38.0,15.5,27.5,21.5,38.5,22.0,39.0,16.0,15.0,28.5,35.0,22.5,24.0,18.0,27.0,24.0,27.5,23.0,19.0,22.0,28.0,15.0,23.5,37.0,23.0,26.5,28.5,23.5,20.5,23.0,20.5,23.0,25.0,18.5,18.0,15.0,25.0,20.5,13.0,18.5,20.5,20.0,26.5,26.0,19.5,20.5,27.5,26.5,21.0,21.0,13.0,23.5,17.5,0.999…_zh.wikipedia.org_all-access_all-agents,0.999…,zh.wikipedia.org,all-access_all-agents
4,45.5,37.5,45.5,52.0,44.0,60.0,104.5,35.0,36.5,105.5,85.0,116.0,52.0,54.5,24.0,50.0,36.0,22.5,30.0,30.0,27.5,24.5,13.0,51.0,12.5,24.5,27.5,12.5,16.0,13.5,19.0,19.0,20.5,28.5,13.5,12.5,10.0,12.5,12.0,17.0,14.5,12.5,15.5,15.5,18.0,25.0,41.0,21.5,22.5,13.0,17.5,27.0,25.0,20.0,23.0,17.5,21.0,13.0,23.5,11.5,16.5,15.5,19.5,1._September_de.wikipedia.org_desktop_all-agents,1._September,de.wikipedia.org,desktop_all-agents


In [98]:
test_all_id = pd.read_csv(path+'key_2.csv')

test_all_id['Date'] = [page[-10:] for page in tqdm(test_all_id.Page)]
test_all_id['Page'] = [page[:-11] for page in tqdm(test_all_id.Page)]
test_all_id.head()

100%|██████████| 8993906/8993906 [00:03<00:00, 2350947.89it/s]


,Page,Id,Date
0,007_スペクター_ja.wikipedia.org_all-access_all-agents,0b293039387a,2017-09-13
1,007_スペクター_ja.wikipedia.org_all-access_all-agents,7114389dd824,2017-09-14
2,007_スペクター_ja.wikipedia.org_all-access_all-agents,057b02ff1f09,2017-09-15
3,007_スペクター_ja.wikipedia.org_all-access_all-agents,bd2aca21caa3,2017-09-16
4,007_スペクター_ja.wikipedia.org_all-access_all-agents,c0effb42cdd5,2017-09-17


In [99]:
test_all = test_all_id.drop('Id', axis=1)
test_all['Visits_true'] = np.NaN

test_all.Visits_true = test_all.Visits_true * offset
test_all = test_all.pivot(index='Page', columns='Date', values='Visits_true').astype('float32').reset_index()

test_all['2017-11-14'] = np.NaN
test_all.sort_values(by='Page', inplace=True)
test_all.reset_index(drop=True, inplace=True)

test_all.head()


Date,Page,2017-09-13,2017-09-14,2017-09-15,2017-09-16,2017-09-17,2017-09-18,2017-09-19,2017-09-20,2017-09-21,2017-09-22,2017-09-23,2017-09-24,2017-09-25,2017-09-26,2017-09-27,2017-09-28,2017-09-29,2017-09-30,2017-10-01,2017-10-02,2017-10-03,2017-10-04,2017-10-05,2017-10-06,2017-10-07,2017-10-08,2017-10-09,2017-10-10,2017-10-11,2017-10-12,2017-10-13,2017-10-14,2017-10-15,2017-10-16,2017-10-17,2017-10-18,2017-10-19,2017-10-20,2017-10-21,2017-10-22,2017-10-23,2017-10-24,2017-10-25,2017-10-26,2017-10-27,2017-10-28,2017-10-29,2017-10-30,2017-10-31,2017-11-01,2017-11-02,2017-11-03,2017-11-04,2017-11-05,2017-11-06,2017-11-07,2017-11-08,2017-11-09,2017-11-10,2017-11-11,2017-11-12,2017-11-13,2017-11-14
0,!vote_en.wikipedia.org_all-access_all-agents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,!vote_en.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,!vote_en.wikipedia.org_desktop_all-agents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_all-agents",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_spider",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
test_all.shape

(145063, 64)

In [101]:
test_all_columns_date = list(test_all.columns[1:])
first_day = 2 # 2017-13-09 is a Wednesday
test_all_columns_code = ['w%d_d%d' % (i // 7, (first_day + i) % 7) for i in range(63)]
cols = ['Page']
cols.extend(test_all_columns_code)
test_all.columns = cols
test_all.head()

,Page,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w0_d0,w0_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w1_d0,w1_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w2_d0,w2_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w3_d0,w3_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w4_d0,w4_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w5_d0,w5_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w6_d0,w6_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w7_d0,w7_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w8_d0,w8_d1
0,!vote_en.wikipedia.org_all-access_all-agents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,!vote_en.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,!vote_en.wikipedia.org_desktop_all-agents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_all-agents",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_spider",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
# test_all = test_all.merge(train_key, how='left', on='Page')
# test_all.head()
test_all = test_all.merge(train_key, on='Page') #default how='inner'
test_all.head()

,Page,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w0_d0,w0_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w1_d0,w1_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w2_d0,w2_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w3_d0,w3_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w4_d0,w4_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w5_d0,w5_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w6_d0,w6_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w7_d0,w7_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w8_d0,w8_d1,PageTitle,Site,AccessAgent
0,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,(1)_Ceres_de.wikipedia.org_desktop_all-agents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,0.999…_zh.wikipedia.org_all-access_all-agents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999…,zh.wikipedia.org,all-access_all-agents
4,1._September_de.wikipedia.org_desktop_all-agents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1._September,de.wikipedia.org,desktop_all-agents


In [103]:
y_cols = test.columns[:63]
y_cols

Index(['w0_d4', 'w0_d5', 'w0_d6', 'w0_d0', 'w0_d1', 'w0_d2', 'w0_d3', 'w1_d4',
       'w1_d5', 'w1_d6', 'w1_d0', 'w1_d1', 'w1_d2', 'w1_d3', 'w2_d4', 'w2_d5',
       'w2_d6', 'w2_d0', 'w2_d1', 'w2_d2', 'w2_d3', 'w3_d4', 'w3_d5', 'w3_d6',
       'w3_d0', 'w3_d1', 'w3_d2', 'w3_d3', 'w4_d4', 'w4_d5', 'w4_d6', 'w4_d0',
       'w4_d1', 'w4_d2', 'w4_d3', 'w5_d4', 'w5_d5', 'w5_d6', 'w5_d0', 'w5_d1',
       'w5_d2', 'w5_d3', 'w6_d4', 'w6_d5', 'w6_d6', 'w6_d0', 'w6_d1', 'w6_d2',
       'w6_d3', 'w7_d4', 'w7_d5', 'w7_d6', 'w7_d0', 'w7_d1', 'w7_d2', 'w7_d3',
       'w8_d4', 'w8_d5', 'w8_d6', 'w8_d0', 'w8_d1', 'w8_d2', 'w8_d3'],
      dtype='object')

In [104]:
for test in tests:
    test.reset_index(inplace=True)
test_all = test_all.reset_index()

In [105]:
test_all.shape

(7253, 68)

In [106]:
test = pd.concat(tests[2:5], axis=0).reset_index(drop=True)
test.shape

(21759, 68)

In [107]:
test.head()

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,5.5,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,1,0.0,2.0,0.0,15.0,1.5,0.0,2.0,2.0,1.0,1.5,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.5,0.5,3.0,0.5,0.5,0.5,1.0,4.0,1.0,0.5,1.5,0.5,0.0,1.0,1.5,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,1.0,1.0,2.5,0.0,1.0,1.0,0.5,0.5,1.5,1.0,0.5,0.5,2.0,2.0,0.0,2.0,0.5,0.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,2,117.5,92.5,90.5,98.5,82.5,71.0,79.5,128.0,87.5,92.5,82.0,111.5,74.0,59.5,89.5,133.5,111.5,85.0,108.5,71.5,154.0,137.0,135.0,104.5,94.0,66.5,64.5,82.5,146.0,93.0,109.5,125.5,85.5,70.5,80.0,90.0,89.5,103.5,98.5,82.0,65.0,85.5,140.5,105.0,92.5,86.0,77.0,61.5,59.5,73.5,75.0,81.5,108.5,125.5,105.5,135.0,145.0,116.0,101.0,130.0,122.0,112.5,140.5,(1)_Ceres_de.wikipedia.org_desktop_all-agents,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,3,21.0,26.0,27.5,15.5,12.0,20.0,20.5,15.5,38.0,15.5,27.5,21.5,38.5,22.0,39.0,16.0,15.0,28.5,35.0,22.5,24.0,18.0,27.0,24.0,27.5,23.0,19.0,22.0,28.0,15.0,23.5,37.0,23.0,26.5,28.5,23.5,20.5,23.0,20.5,23.0,25.0,18.5,18.0,15.0,25.0,20.5,13.0,18.5,20.5,20.0,26.5,26.0,19.5,20.5,27.5,26.5,21.0,21.0,13.0,23.5,17.5,24.0,18.0,0.999…_zh.wikipedia.org_all-access_all-agents,0.999…,zh.wikipedia.org,all-access_all-agents
4,4,45.5,52.0,44.0,60.0,104.5,35.0,36.5,105.5,85.0,116.0,52.0,54.5,24.0,50.0,36.0,22.5,30.0,30.0,27.5,24.5,13.0,51.0,12.5,24.5,27.5,12.5,16.0,13.5,19.0,19.0,20.5,28.5,13.5,12.5,10.0,12.5,12.0,17.0,14.5,12.5,15.5,15.5,18.0,25.0,41.0,21.5,22.5,13.0,17.5,27.0,25.0,20.0,23.0,17.5,21.0,13.0,23.5,11.5,16.5,15.5,19.5,12.5,21.0,1._September_de.wikipedia.org_desktop_all-agents,1._September,de.wikipedia.org,desktop_all-agents


In [108]:
test.tail()

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
21754,7248,153.5,231.0,138.0,318.0,336.5,273.5,251.5,197.0,175.0,305.5,308.5,287.5,194.5,190.5,144.5,113.0,154.0,130.5,99.5,118.5,297.0,134.5,136.5,96.5,117.0,100.0,94.0,114.0,101.0,99.0,84.0,104.0,106.5,108.0,101.0,105.0,148.0,82.0,107.0,97.0,80.0,90.0,94.5,119.0,120.0,131.0,83.0,72.5,117.5,120.5,126.5,125.0,79.5,116.5,81.5,119.5,154.0,195.0,169.0,140.5,175.0,120.0,136.5,龍が如く6_命の詩。_ja.wikipedia.org_desktop_all-agents,龍が如く6_命の詩。,ja.wikipedia.org,desktop_all-agents
21755,7249,5.5,6.0,4.0,4.5,3.0,5.0,6.0,3.0,3.0,5.5,2.5,4.0,3.5,5.0,2.5,3.0,2.0,2.5,2.5,3.0,2.5,3.5,1.5,3.5,5.0,2.0,4.5,4.5,4.5,2.0,2.0,6.5,4.0,3.5,4.0,3.5,4.0,2.5,4.5,3.0,2.5,1.5,3.5,4.5,2.0,3.5,1.5,1.5,7.5,5.0,2.5,5.0,5.0,2.0,3.5,11.5,2.5,2.5,4.5,1.0,3.0,3.5,4.5,龍千玉_zh.wikipedia.org_all-access_spider,龍千玉,zh.wikipedia.org,all-access_spider
21756,7250,35.0,46.5,57.0,64.0,49.0,60.0,56.0,26.0,24.5,54.0,54.5,48.0,69.5,38.5,48.5,30.0,30.0,47.0,32.0,43.0,43.0,48.5,37.0,23.5,36.0,65.5,38.5,47.0,89.0,90.0,111.5,106.5,78.5,86.0,84.5,345.5,462.5,95.5,89.5,168.5,115.5,108.0,267.0,255.5,324.0,277.0,335.5,283.5,231.0,213.5,202.5,250.5,254.5,444.0,381.5,287.5,353.0,303.5,177.0,180.0,215.5,173.0,264.5,龔嘉欣_zh.wikipedia.org_mobile-web_all-agents,龔嘉欣,zh.wikipedia.org,mobile-web_all-agents
21757,7251,6.5,5.0,3.0,5.0,6.5,4.5,6.0,2.0,2.0,4.5,6.0,4.0,4.0,4.5,30.5,36.0,5.0,4.0,7.5,3.0,4.0,5.0,7.0,14.0,7.5,8.5,4.5,6.5,13.0,6.5,20.5,22.0,7.5,10.0,4.0,1.0,5.0,7.0,8.5,10.0,3.5,5.0,8.5,7.5,3.5,6.5,6.5,5.0,7.5,4.0,4.0,4.5,4.0,6.0,4.5,0.5,2.0,3.5,4.5,4.5,4.0,5.0,2.0,龔照勝_zh.wikipedia.org_desktop_all-agents,龔照勝,zh.wikipedia.org,desktop_all-agents
21758,7252,97.0,105.5,54.5,72.5,74.5,87.5,96.0,85.0,82.0,85.5,83.5,95.0,160.0,84.0,76.5,83.0,105.5,95.5,105.0,163.0,74.0,90.5,109.5,97.0,86.5,112.0,84.5,100.0,119.0,87.5,106.5,116.5,131.0,79.5,67.0,102.5,91.0,100.0,81.5,68.5,67.0,77.0,110.0,113.5,88.5,76.0,116.5,83.0,99.0,117.0,119.0,101.0,112.0,93.0,81.5,81.0,110.0,92.5,85.0,76.5,85.5,78.0,95.5,대문_commons.wikimedia.org_all-access_all-agents,대문,commons.wikimedia.org,all-access_all-agents


In [109]:
test[test['index']==0]


,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,5.5,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
7253,0,6.5,13.0,12.5,39.0,10.0,6.0,15.0,6.0,10.5,12.0,6.5,8.0,13.0,7.5,10.0,8.5,4.0,10.0,8.5,13.0,7.5,12.5,8.5,6.5,15.0,8.0,7.5,9.5,5.0,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,7.5,10.5,46.5,7.5,10.5,5.5,16.0,8.5,8.5,17.0,6.0,6.0,8.5,11.5,17.0,14.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
14506,0,6.5,10.5,12.5,39.0,10.0,6.0,15.0,6.0,8.5,12.0,6.5,8.0,13.0,7.5,10.0,8.5,4.0,10.0,8.5,13.0,7.5,12.5,22.5,6.5,15.0,8.0,7.5,9.5,5.0,9.0,17.0,6.5,12.5,23.0,9.0,5.0,10.5,11.5,10.0,6.0,8.5,10.0,7.5,8.5,46.5,7.5,10.5,5.5,16.0,8.5,14.5,17.0,6.0,6.0,8.5,11.5,17.0,12.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents


In [110]:
test_all = test_all[test.columns].copy()
test_all.head()

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(1)_Ceres_de.wikipedia.org_desktop_all-agents,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999…_zh.wikipedia.org_all-access_all-agents,0.999…,zh.wikipedia.org,all-access_all-agents
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1._September_de.wikipedia.org_desktop_all-agents,1._September,de.wikipedia.org,desktop_all-agents


In [111]:
train_cols = ['d_%d' % i for i in range(train_norms[0].shape[1])]
len(train_cols)

181

In [112]:
for train_norm in train_norms:
    train_norm.columns = train_cols
train_all_norm.columns = train_cols
train_norm = pd.concat(train_norms[2:5], axis=0).reset_index(drop=True)
train_norm.shape

(21759, 181)

In [113]:
train_norm.head()


,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180
0,2.197225,2.397895,2.140066,2.708050,2.602690,2.674149,2.639057,2.442347,2.602690,2.525729,2.740840,2.397895,2.740840,2.397895,2.639057,2.772589,3.157001,2.995732,1.871802,2.140066,2.602690,2.484907,2.251292,1.945910,2.484907,2.564949,2.639057,2.970414,2.772589,2.803360,2.833213,2.525729,2.708050,2.995732,2.525729,2.140066,3.650658,2.397895,2.484907,2.833213,2.639057,2.442347,2.397895,2.302585,2.772589,2.833213,2.197225,2.803360,2.862201,2.803360,2.740840,4.110874,2.708050,2.708050,2.602690,2.302585,2.397895,2.970414,2.351375,2.944439,2.602690,2.740840,2.639057,2.772589,2.602690,2.564949,2.564949,3.597312,2.602690,2.302585,2.525729,2.772589,2.602690,2.442347,2.397895,2.397895,2.484907,2.708050,2.397895,2.564949,2.833213,2.442347,2.302585,2.639057,2.890372,2.995732,2.708050,2.525729,2.917771,2.525729,2.351375,2.442347,2.351375,2.014903,2.197225,2.397895,2.079442,2.484907,2.251292,2.351375,2.564949,2.197225,2.602690,2.442347,2.302585,2.197225,2.397895,2.014903,1.945910,2.397895,3.068053,2.351375,2.639057,2.197225,1.791759,2.397895,2.197225,2.251292,2.251292,3.332205,2.484907,2.740840,2.525729,2.251292,2.442347,2.197225,1.791759,2.484907,2.014903,2.442347,2.525729,2.484907,2.639057,2.079442,2.302585,2.140066,2.251292,2.917771,2.251292,2.995732,2.674149,1.704748,1.704748,1.871802,2.442347,2.70805,2.772589,2.995732,2.833213,2.833213,2.970414,3.178054,3.044523,3.020425,3.135494,3.068053,2.995732,2.995732,2.862201,2.833213,1.609438,2.74084,2.014903,2.397895,2.251292,2.197225,2.674149,1.791759,2.564949,2.014903,2.140066,1.791759,4.442651,2.564949,2.140066,2.140066,1.609438,2.014903,2.197225,2.970414,2.014903
1,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,0.405465,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,0.405465,NaN,0.405465,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.405465,0.000000
2,1.252763,0.405465,2.602690,3.314186,2.140066,2.197225,1.386294,1.504077,4.007333,4.212128,1.609438,2.708050,2.079442,2.674149,0.916291,1.098612,1.504077,1.098612,0.693147,2.251292,1.386294,3.113515,2.140066,2.397895,0.693147,2.251292,3.068053,1.791759,1.098612,1.609438,2.302585,1.252763,1.945910,1.098612,0.693147,1.945910,1.504077,1.504077,0.405465,1.098612,1.704748,3.295837,3.540959,1.504077,2.525729,1.609438,1.386294,2.525729,1.871802,1.386294,2.302585,1.504077,1.098612,1.791759,1.386294,2.564949,3.332205,1.098612,2.197225,1.504

In [114]:
train_norm_diff = train_norm - train_norm.shift(-1, axis=1)
train_norm_diff.head()

train_all_norm_diff = train_all_norm - train_all_norm.shift(-1, axis=1)
train_all_norm_diff.head()

train_norm_diff7 = train_norm - train_norm.shift(-7, axis=1)
train_norm_diff7.head()

train_all_norm_diff7 = train_all_norm - train_all_norm.shift(-7, axis=1)
train_all_norm_diff7.head()

,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180
0,-0.300105,0.251314,-0.062521,1.143064,0.916291,0.325423,-0.192372,-0.287682,0.028988,-0.029853,-1.473306,-0.936093,-0.503104,0.287682,0.448025,-0.162519,-0.385663,1.067841,0.466090,0.428996,-0.236389,-1.023389,-0.117783,0.198451,0.451985,-0.117783,-0.930475,-1.486378,-0.475846,-1.317301,-0.940388,-0.090972,-0.105361,-0.259109,0.379032,1.376632,2.179525,1.518466,-0.265703,-1.273965,0.910871,0.774640,0.313658,0.305382,0.302281,-0.095310,1.407497,-0.378066,-0.683668,-0.191055,-0.579818,-0.211309,0.277632,0.377294,1.044545,1.252763,0.302281,-0.444686,-0.716678,-0.770108,-0.080043,-0.456758,0.105361,-1.078810,-0.050010,-0.349674,0.656780,0.039221,0.567984,0.523248,0.579818,0.455476,1.066351,0.074108,-0.732368,-0.385663,-0.362906,-0.693147,-0.238411,-0.251314,-0.455476,0.550046,-0.418710,-0.419854,1.029619,0.095310,-0.071459,-0.590493,0.182322,0.054067,0.464306,-0.559616,-0.154151,-0.321584,0.870078,-0.076961,0.287682,-0.670158,0.722135,0.377294,0.510826,0.543615,0.300105,0.076961,0.765468,-0.424883,-0.348307,-0.510826,-0.405465,-0.470004,-0.751416,-0.854415,0.122602,0.530628,1.203973,0.204794,0.169899,1.079920,1.077559,-0.083382,-0.371563,-0.980829,-0.572519,0.351398,-0.367725,-0.523248,0.223144,-0.243622,-0.030772,1.098612,-0.456758,-0.479573,0.251314,-0.095310,0.000000,0.238411,-1.045968,0.310155,1.098612,-0.322773,-0.715620,0.666479,0.262364,0.666479,-0.167054,-0.728238,0.109199,0.280302,-0.521297,0.000000,-0.387766,-0.916291,-0.098440,0.000000,-0.111226,0.064538,-0.470004,-0.356675,1.083345,0.575364,0.860201,0.593064,-0.154151,0.169899,0.744441,-0.127833,-0.635989,-1.098612,-0.805625,-0.721318,-0.169899,-0.552069,-0.542324,-0.385663,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,0.000000,NaN,NaN,-1.252763,NaN,NaN,NaN,-0.980829,NaN,0.287682,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,-0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.510826,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.446919,0.133531,-0.747214,-0.663294,0.385663,-0.251314,-0.117783,-0.257829,-0.538996,-0.947381,-0.087011,-0.777704,0.810930,1.098612,0.451985,1.098612,1.183770,1.280934,1.126011,-0.117783,-0.510826,-0.887303,-0.318454,0.143101,0.223144,0.087011,-0.635989,-0.470004,0.125163,-1.036092,0.167054,-0.559616,-1.339774,0.348307,0.000000,0.628609,-0.203599,-0.167054,0.074108,0.847298,-1.011601,0.980829,-0.405465,1.691676,-0.143101,-0.961411,-0.916291,0.318454,-1.021651,1.386294,-0.693147,0.405465,0.961411,0.973449,0.980829,0.510826,-1.504077,-0.61

In [115]:
train_norm = train_norm.iloc[:,::-1]
train_norm_diff7m = train_norm - train_norm.rolling(window=7, axis=1).median()
train_norm = train_norm.iloc[:,::-1]
train_norm_diff7m = train_norm_diff7m.iloc[:,::-1]
train_norm_diff7m.head()

,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180
0,-0.405465,-0.204794,-0.462624,0.105361,0.000000,0.071459,0.036368,-0.083382,0.000000,-0.113329,0.000000,-0.342945,0.000000,-0.241162,0.000000,0.169899,0.672094,0.744441,-0.379490,-0.344841,0.117783,0.000000,-0.313658,-0.693147,-0.287682,-0.207639,-0.133531,0.167054,0.000000,0.095310,0.125163,0.000000,0.182322,0.470004,0.000000,-0.344841,1.165751,-0.044452,0.000000,0.194156,0.196710,0.000000,-0.374693,-0.500775,-0.030772,0.029853,-0.606136,0.000000,0.121361,0.095310,0.032790,1.402823,0.105361,0.105361,0.000000,-0.300105,-0.241162,0.229574,-0.287682,0.305382,0.000000,0.101783,0.036368,0.169899,0.037740,0.000000,-0.037740,0.994622,0.076961,-0.139762,0.040822,0.287682,0.160343,0.000000,-0.087011,-0.087011,0.000000,0.143101,-0.167054,-0.074108,0.125163,-0.196710,-0.405465,-0.068993,0.182322,0.470004,0.182322,0.083382,0.566395,0.174353,0.00000,0.090972,0.100084,-0.236389,-0.154151,0.046520,-0.271934,0.042560,-0.100084,0.000000,0.167054,-0.105361,0.300105,0.139762,0.000000,-0.154151,0.000000,-0.336472,-0.405465,0.000000,0.716678,0.100084,0.387766,-0.054067,-0.459532,0.000000,-0.287682,-0.233615,-0.233615,0.847298,0.042560,0.298493,0.274437,0.000000,0.000000,-0.245122,-0.693147,0.000000,-0.427444,0.000000,0.223144,0.182322,0.387766,-0.171850,0.000000,-0.111226,0.000000,0.666479,0.000000,0.553385,0.231802,-0.737599,-1.003302,-0.900787,-0.390866,-0.125163,-0.197826,0.0,-0.187212,-0.211309,-0.074108,0.133531,0.024098,0.024693,0.139762,0.205852,0.162519,0.254892,0.464306,0.581922,-0.641854,0.489548,-0.236389,0.146604,0.054067,0.057158,0.534082,-0.348307,0.424883,-0.125163,0.000000,-0.348307,2.302585,0.424883,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.887303,-1.734601,0.405465,1.116961,0.000000,0.000000,-0.693147,-1.170071,1.333185,2.132686,0.000000,1.203973,0.980829,1.575536,-0.182322,-0.287682,0.000000,-1.041454,-1.446919,0.000000,-0.864997,0.862224,0.000000,0.606136,-1.098612,0.459532,1.276294,0.182322,-0.154151,0.000000,0.798508,-0.251314,0.441833,0.000000,-0.810930,0.441833,0.000000,0.000000,-1.299283,-0.606136,0.000000,0.770108,1.669157,-0.105361,0.653926,0.000000,-0.117783,0.733969,0.367725,-0.117783,0.510826,0.000000,-0.693147,0.000000,-0.810930,0.367725,1.134980,-0.510826,0.000000,-0.747214,0.000000,-0.587787,0.773190,-1.734601,0.777704,0.955511,0.587787,0.530628,

In [116]:
train_all_norm = train_all_norm.iloc[:,::-1]
train_all_norm_diff7m = train_all_norm - train_all_norm.rolling(window=7, axis=1).median()
train_all_norm = train_all_norm.iloc[:,::-1]
train_all_norm_diff7m = train_all_norm_diff7m.iloc[:,::-1]
train_all_norm_diff7m.head()

,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180
0,-0.587787,0.223144,-0.121361,0.678758,0.415515,0.087011,0.200671,0.000000,0.058841,0.000000,-0.435318,-0.530628,-0.325423,0.000000,0.000000,0.000000,-0.162519,0.803148,0.242946,0.264693,-0.064538,-0.361013,0.051293,0.274437,-0.141078,-0.171850,-0.305382,-0.075986,0.352221,0.000000,-0.549108,-1.590206,-1.051210,-0.372049,0.489235,0.000000,0.602175,0.965081,-0.122602,0.287682,1.120591,1.343735,-0.143101,-0.456758,-0.265703,0.000000,1.466337,0.114410,0.473784,-0.552069,-0.857450,-0.663294,0.000000,0.336472,0.770108,1.475906,-0.042560,0.040822,-0.133531,0.000000,-0.223144,-0.456758,0.000000,-0.567984,0.000000,0.047628,0.275412,-0.143101,0.000000,-0.036368,0.579818,0.381367,0.852777,0.113329,0.000000,-0.424883,-0.485508,0.074108,0.000000,-0.213574,-0.037740,0.454736,-0.182322,-0.361013,0.470004,0.000000,-0.105361,0.312375,0.000000,0.236389,0.154151,-0.371563,0.000000,-0.188052,0.748717,-0.336472,0.028171,-0.342945,0.000000,0.121361,0.393043,0.138150,0.117783,0.117783,0.583146,-0.162519,0.000000,0.000000,-0.328504,-0.262364,-0.076961,-0.470004,-0.268264,0.000000,0.223144,0.000000,0.330242,0.834798,0.759105,0.139762,0.000000,-0.606136,-0.127833,0.000000,-0.328504,-0.485508,-0.076961,0.071459,0.169899,0.367725,-0.313658,-0.037740,-0.105361,-0.405465,0.209720,0.095310,-0.693147,0.000000,0.479573,-0.046520,-0.167054,0.243622,0.000000,0.519875,0.095310,-0.356675,0.109199,0.548566,-0.313658,-0.262364,-0.313658,-0.074108,0.000000,-0.207639,0.060625,0.000000,-0.405465,-0.133531,0.708651,0.000000,-0.143101,0.236389,0.310155,0.374693,0.597837,0.046520,-0.154151,-0.820981,-0.251314,0.058841,-0.200671,-0.552069,-0.277632,0.000000,-0.264693,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.559616,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.386294,-0.060625,-0.635989,0.000000,0.274437,-0.194156,0.287682,-0.057158,-0.305382,-0.146604,0.318454,-0.344841,-0.405465,0.000000,-0.087011,0.000000,1.252763,0.944462,1.308333,-0.223144,-0.105361,0.154151,-0.405465,0.223144,-0.182322,0.000000,-0.200671,0.143101,0.961411,-0.167054,-0.268264,-0.753772,-0.435318,-0.344841,0.000000,0.223144,0.661398,-0.374693,-0.133531,0.965081,-0.287682,0.287682,0.000000,0.747214,0.000000,-0.143101,0.000000,0.318454,-0.980829,0.000000,-1.232144,-0.470004,0.348307,1.167605,0.538996,0.653926,-0.773190,0.000000,-0.405465,-0.268264,0.000000,-0.510826,0.000000,0.730888,0

In [117]:
test_all.head(2)

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider


In [118]:
test.head(2)

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,5.5,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,1,0.0,2.0,0.0,15.0,1.5,0.0,2.0,2.0,1.0,1.5,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.5,0.5,3.0,0.5,0.5,0.5,1.0,4.0,1.0,0.5,1.5,0.5,0.0,1.0,1.5,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,1.0,1.0,2.5,0.0,1.0,1.0,0.5,0.5,1.5,1.0,0.5,0.5,2.0,2.0,0.0,2.0,0.5,0.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider


In [119]:
test.head(10)

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,5.5,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,1,0.0,2.0,0.0,15.0,1.5,0.0,2.0,2.0,1.0,1.5,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.5,0.5,3.0,0.5,0.5,0.5,1.0,4.0,1.0,0.5,1.5,0.5,0.0,1.0,1.5,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,1.0,1.0,2.5,0.0,1.0,1.0,0.5,0.5,1.5,1.0,0.5,0.5,2.0,2.0,0.0,2.0,0.5,0.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,2,117.5,92.5,90.5,98.5,82.5,71.0,79.5,128.0,87.5,92.5,82.0,111.5,74.0,59.5,89.5,133.5,111.5,85.0,108.5,71.5,154.0,137.0,135.0,104.5,94.0,66.5,64.5,82.5,146.0,93.0,109.5,125.5,85.5,70.5,80.0,90.0,89.5,103.5,98.5,82.0,65.0,85.5,140.5,105.0,92.5,86.0,77.0,61.5,59.5,73.5,75.0,81.5,108.5,125.5,105.5,135.0,145.0,116.0,101.0,130.0,122.0,112.5,140.5,(1)_Ceres_de.wikipedia.org_desktop_all-agents,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,3,21.0,26.0,27.5,15.5,12.0,20.0,20.5,15.5,38.0,15.5,27.5,21.5,38.5,22.0,39.0,16.0,15.0,28.5,35.0,22.5,24.0,18.0,27.0,24.0,27.5,23.0,19.0,22.0,28.0,15.0,23.5,37.0,23.0,26.5,28.5,23.5,20.5,23.0,20.5,23.0,25.0,18.5,18.0,15.0,25.0,20.5,13.0,18.5,20.5,20.0,26.5,26.0,19.5,20.5,27.5,26.5,21.0,21.0,13.0,23.5,17.5,24.0,18.0,0.999…_zh.wikipedia.org_all-access_all-agents,0.999…,zh.wikipedia.org,all-access_all-agents
4,4,45.5,52.0,44.0,60.0,104.5,35.0,36.5,105.5,85.0,116.0,52.0,54.5,24.0,50.0,36.0,22.5,30.0,30.0,27.5,24.5,13.0,51.0,12.5,24.5,27.5,12.5,16.0,13.5,19.0,19.0,20.5,28.5,13.5,12.5,10.0,12.5,12.0,17.0,14.5,12.5,15.5,15.5,18.0,25.0,41.0,21.5,22.5,13.0,17.5,27.0,25.0,20.0,23.0,17.5,21.0,13.0,23.5,11.5,16.5,15.5,19.5,12.5,21.0,1._September_de.wikipedia.org_desktop_all-agents,1._September,de.wikipedia.org,desktop_all-agents
5,5,140.0,181.5,116.0,123.0,168.0,355.5,134.0,113.0,105.5,119.5,110.5,81.5,81.0,406.0,234.5,163.0,125.5,116.5,91.5,117.0,83.0,91.0,89.0,114.0,103.0,197.5,150.0,83.0,100.0,104.5,94.5,92.0,125.5,86.0,101.5,162.0,129.0,83.5,80.0,68.5,79.0,73.0,93.5,94.5,89.5,82.0,106.5,63.5,64.5,61.5,78.0,91.0,97.0,70.5,60.0,62.0,86.0,113.0,62.0,137.0,84.5,58.5,84.5,100メートル競走_ja.wikipedia.org_desktop_all-agents,100メートル競走,ja.wikipedia.org,desktop_all-agents
6,6,12.5,7.5,10.0,8.0,10.0,11.5,12.5,16.5,22.5,41.5,42.0,32.0,21.0,31.0,34.5,22.5,21.5,41.0,34.0,39.5,36.0,44.5,32.5,47.5,48.5,49.5,40.0,48.0,55.5,46.0,67.0,65.0,70.5,61.5,76.5,81.0,75.5,65.0,87.5,95.0,94.0,113.0,351.5,365.0,244.5,311.0,506.0,466.5,998.5,1375.5,170.0,53.0,35.0,21.0,21.0,22.5,17.5,12.0,8.5,16.5,16.5,8.0,6.5,10月31日_zh.wikipedia.org_mobile-web_all-agents,10月31日,zh.wikipedia.org,mobile-web_all-agents
7,7,289.0,281.0,294.0,285.5,264.5,206.0,255.5,276.0,335.0,280.5,268.0,259.0,266.0,277.0,344.5,367.0,328.0,374.0,406.0,374.0,350.5,480.0,432.5,453.0,439.0,394.5,332.5,302.0,375.0,483.5,493.0,505.0,487.5,386.0,412.5,597.5,611.0,611.5,660.5,629.5,514.5,546.0,814.5,874.0,937.0,1014.5,976.0,808.5,1330.0,4647.0,4202.0,1944.0,1446.5,1252.0,957.0,1053.0,1204.0,1076.0,899.5,956.5,884.5,780.5,756.0,11月_ja.wikipedia.org_all-access_all-agents,11月,ja.wikipedia.org,all-access_all-agents
8,8,49.5,41.0,38.5,40.0,44.0,52.0,60.5,37.0,29.0,36.0,26.5,43.5,39.5,65.0,26.5,54.0,35.5,47.5,43.5,54.

In [120]:
test_all.head(10)

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(1)_Ceres_de.wikipedia.org_desktop_all-agents,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999…_zh.wikipedia.org_all-access_all-agents,0.999…,zh.wikipedia.org,all-access_all-agents
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1._September_de.wikipedia.org_desktop_all-agents,1._September,de.wikipedia.org,desktop_all-agents
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100メートル競走_ja.wikipedia.org_desktop_all-agents,100メートル競走,ja.wikipedia.org,desktop_all-agents
6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10月31日_zh.wikipedia.org_mobile-web_all-agents,10月31日,zh.wikipedia.org,mobile-web_all-agents
7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11月_ja.wikipedia.org_all-access_all-agents,11月,ja.wikipedia.org,all-access_all-agents
8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127_heures_fr.wikipedia.org_mobile-web_all-agents,127_heures,fr.wikipedia.org,mobile-web_all-agents
9,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12_de_octubre_es.wikipedia.org_mobile-w

In [121]:
all(test[:test_all.shape[0]].Page == test_all.Page)

True

In [122]:
test_all.shape

(7253, 68)

In [123]:
test

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent
0,0,5.5,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents
1,1,0.0,2.0,0.0,15.0,1.5,0.0,2.0,2.0,1.0,1.5,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.5,0.5,3.0,0.5,0.5,0.5,1.0,4.0,1.0,0.5,1.5,0.5,0.0,1.0,1.5,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,1.0,1.0,2.5,0.0,1.0,1.0,0.5,0.5,1.5,1.0,0.5,0.5,2.0,2.0,0.0,2.0,0.5,0.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider
2,2,117.5,92.5,90.5,98.5,82.5,71.0,79.5,128.0,87.5,92.5,82.0,111.5,74.0,59.5,89.5,133.5,111.5,85.0,108.5,71.5,154.0,137.0,135.0,104.5,94.0,66.5,64.5,82.5,146.0,93.0,109.5,125.5,85.5,70.5,80.0,90.0,89.5,103.5,98.5,82.0,65.0,85.5,140.5,105.0,92.5,86.0,77.0,61.5,59.5,73.5,75.0,81.5,108.5,125.5,105.5,135.0,145.0,116.0,101.0,130.0,122.0,112.5,140.5,(1)_Ceres_de.wikipedia.org_desktop_all-agents,(1)_Ceres,de.wikipedia.org,desktop_all-agents
3,3,21.0,26.0,27.5,15.5,12.0,20.0,20.5,15.5,38.0,15.5,27.5,21.5,38.5,22.0,39.0,16.0,15.0,28.5,35.0,22.5,24.0,18.0,27.0,24.0,27.5,23.0,19.0,22.0,28.0,15.0,23.5,37.0,23.0,26.5,28.5,23.5,20.5,23.0,20.5,23.0,25.0,18.5,18.0,15.0,25.0,20.5,13.0,18.5,20.5,20.0,26.5,26.0,19.5,20.5,27.5,26.5,21.0,21.0,13.0,23.5,17.5,24.0,18.0,0.999…_zh.wikipedia.org_all-access_all-agents,0.999…,zh.wikipedia.org,all-access_all-agents
4,4,45.5,52.0,44.0,60.0,104.5,35.0,36.5,105.5,85.0,116.0,52.0,54.5,24.0,50.0,36.0,22.5,30.0,30.0,27.5,24.5,13.0,51.0,12.5,24.5,27.5,12.5,16.0,13.5,19.0,19.0,20.5,28.5,13.5,12.5,10.0,12.5,12.0,17.0,14.5,12.5,15.5,15.5,18.0,25.0,41.0,21.5,22.5,13.0,17.5,27.0,25.0,20.0,23.0,17.5,21.0,13.0,23.5,11.5,16.5,15.5,19.5,12.5,21.0,1._September_de.wikipedia.org_desktop_all-agents,1._September,de.wikipedia.org,desktop_all-agents
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21754,7248,153.5,231.0,138.0,318.0,336.5,273.5,251.5,197.0,175.0,305.5,308.5,287.5,194.5,190.5,144.5,113.0,154.0,130.5,99.5,118.5,297.0,134.5,136.5,96.5,117.0,100.0,94.0,114.0,101.0,99.0,84.0,104.0,106.5,108.0,101.0,105.0,148.0,82.0,107.0,97.0,80.0,90.0,94.5,119.0,120.0,131.0,83.0,72.5,117.5,120.5,126.5,125.0,79.5,116.5,81.5,119.5,154.0,195.0,169.0,140.5,175.0,120.0,136.5,龍が如く6_命の詩。_ja.wikipedia.org_desktop_all-agents,龍が如く6_命の詩。,ja.wikipedia.org,desktop_all-agents
21755,7249,5.5,6.0,4.0,4.5,3.0,5.0,6.0,3.0,3.0,5.5,2.5,4.0,3.5,5.0,2.5,3.0,2.0,2.5,2.5,3.0,2.5,3.5,1.5,3.5,5.0,2.0,4.5,4.5,4.5,2.0,2.0,6.5,4.0,3.5,4.0,3.5,4.0,2.5,4.5,3.0,2.5,1.5,3.5,4.5,2.0,3.5,1.5,1.5,7.5,5.0,2.5,5.0,5.0,2.0,3.5,11.5,2.5,2.5,4.5,1.0,3.0,3.5,4.5,龍千玉_zh.wikipedia.org_all-access_spider,龍千玉,zh.wikipedia.org,all-access_spider
21756,7250,35.0,46.5,57.0,64.0,49.0,60.0,56.0,26.0,24.5,54.0,54.5,48.0,69.5,38.5,48.5,30.0,30.0,47.0,32.0,43.0,43.0,48.5,37.0,23.5,36.0,65.5,38.5,47.0,89.0,90.0,111.5,106.5,78.5,86.0,84.5,345.5,462.5,95.5,89.5,168.5,115.5,108.0,267.0,255.5,324.0,277.0,335.5,283.5,231.0,213.5,202.5,250.5,254.5,444.0,381.5,287.5,353.0,303.5,177.0,180.0,215.5,173.0,264

In [124]:
test[:test_all.shape[0]].Page

0       "Keep_me_logged_in"_extended_to_one_year_www.mediawiki.org_all-access_all-agents
1           "Keep_me_logged_in"_extended_to_one_year_www.mediawiki.org_all-access_spider
2                                          (1)_Ceres_de.wikipedia.org_desktop_all-agents
3                                          0.999…_zh.wikipedia.org_all-access_all-agents
4                                       1._September_de.wikipedia.org_desktop_all-agents
                                              ...                                       
7248                                      龍が如く6_命の詩。_ja.wikipedia.org_desktop_all-agents
7249                                              龍千玉_zh.wikipedia.org_all-access_spider
7250                                          龔嘉欣_zh.wikipedia.org_mobile-web_all-agents
7251                                             龔照勝_zh.wikipedia.org_desktop_all-agents
7252                                      대문_commons.wikimedia.org_all-access_all-agents
Name: Page, Length: 7

In [125]:
test_all.Page

0       "Keep_me_logged_in"_extended_to_one_year_www.mediawiki.org_all-access_all-agents
1           "Keep_me_logged_in"_extended_to_one_year_www.mediawiki.org_all-access_spider
2                                          (1)_Ceres_de.wikipedia.org_desktop_all-agents
3                                          0.999…_zh.wikipedia.org_all-access_all-agents
4                                       1._September_de.wikipedia.org_desktop_all-agents
                                              ...                                       
7248                                      龍が如く6_命の詩。_ja.wikipedia.org_desktop_all-agents
7249                                              龍千玉_zh.wikipedia.org_all-access_spider
7250                                          龔嘉欣_zh.wikipedia.org_mobile-web_all-agents
7251                                             龔照勝_zh.wikipedia.org_desktop_all-agents
7252                                      대문_commons.wikimedia.org_all-access_all-agents
Name: Page, Length: 7

In [126]:
sites = train_key.Site.unique()
sites

array(['zh.wikipedia.org', 'fr.wikipedia.org', 'en.wikipedia.org',
       'commons.wikimedia.org', 'ru.wikipedia.org', 'www.mediawiki.org',
       'de.wikipedia.org', 'ja.wikipedia.org', 'es.wikipedia.org'],
      dtype=object)

In [127]:
test_site = pd.factorize(test.Site)[0]
test['Site_label'] = test_site
test_all['Site_label'] = test_site[:test_all.shape[0]]

In [128]:
accesses = train_key.AccessAgent.unique()
accesses

array(['all-access_spider', 'desktop_all-agents', 'mobile-web_all-agents',
       'all-access_all-agents'], dtype=object)

In [129]:
test_access = pd.factorize(test.AccessAgent)[0]
test['Access_label'] = test_access
test_all['Access_label'] = test_access[:test_all.shape[0]]

In [130]:
test.shape

(21759, 70)

In [131]:
test.head(2)

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent,Site_label,Access_label
0,0,5.5,13.0,12.5,39.0,10.0,6.0,15.0,6.5,10.5,12.0,6.5,8.0,13.0,7.5,6.0,8.5,4.0,10.0,8.5,13.0,7.5,10.0,8.5,6.5,15.0,8.0,7.5,9.5,12.5,22.5,17.0,6.5,12.5,23.0,9.0,5.0,9.0,11.5,10.0,6.0,8.5,10.0,5.0,10.5,46.5,7.5,10.5,5.5,16.0,7.5,8.5,17.0,6.0,6.0,8.5,11.5,8.5,14.5,15.0,10.0,13.0,7.0,13.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents,0,0
1,1,0.0,2.0,0.0,15.0,1.5,0.0,2.0,2.0,1.0,1.5,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.5,0.5,3.0,0.5,0.5,0.5,1.0,4.0,1.0,0.5,1.5,0.5,0.0,1.0,1.5,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,1.0,1.0,2.5,0.0,1.0,1.0,0.5,0.5,1.5,1.0,0.5,0.5,2.0,2.0,0.0,2.0,0.5,0.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider,0,1


In [132]:
test_all.shape


(7253, 70)

In [133]:
test_all.head(2)

,index,w0_d0,w0_d1,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,Page,PageTitle,Site,AccessAgent,Site_label,Access_label
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_all-agents,0,0
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider","""Keep_me_logged_in""_extended_to_one_year",www.mediawiki.org,all-access_spider,0,1


In [134]:
test0 = test.copy()
test_all0 = test_all.copy()

In [135]:
y_norm_cols = [c+'_norm' for c in y_cols]
y_pred_cols = [c+'_pred' for c in y_cols]

In [136]:
# all visits is median
def add_median(test, train, train_diff, train_diff7, train_diff7m,
               train_key, periods, max_periods, first_train_weekday):
    train =  train.iloc[:,:7*max_periods]
    
    df = train_key[['Page']].copy()
    df['AllVisits'] = train.median(axis=1).fillna(0)
    test = test.merge(df, how='left', on='Page', copy=False)
    test.AllVisits = test.AllVisits.fillna(0).astype('float32')
    
    for site in sites:
        test[site] = (1 * (test.Site == site)).astype('float32')
    
    for access in accesses:
        test[access] = (1 * (test.AccessAgent == access)).astype('float32')

    for (w1, w2) in periods:
        
        df = train_key[['Page']].copy()
        c = 'median_%d_%d' % (w1, w2)
        cm = 'mean_%d_%d' % (w1, w2)
        cmax = 'max_%d_%d' % (w1, w2)
        cd = 'median_diff_%d_%d' % (w1, w2)
        cd7 = 'median_diff7_%d_%d' % (w1, w2)
        cd7m = 'median_diff7m_%d_%d' % (w1, w2)
        cd7mm = 'mean_diff7m_%d_%d' % (w1, w2)
        df[c] = train.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cm] = train.iloc[:,7*w1:7*w2].mean(axis=1, skipna=True) 
        df[cmax] = train.iloc[:,7*w1:7*w2].max(axis=1, skipna=True) 
        df[cd] = train_diff.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cd7] = train_diff7.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cd7m] = train_diff7m.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cd7mm] = train_diff7m.iloc[:,7*w1:7*w2].mean(axis=1, skipna=True) 
        test = test.merge(df, how='left', on='Page', copy=False)
        test[c] = (test[c] - test.AllVisits).fillna(0).astype('float32')
        test[cm] = (test[cm] - test.AllVisits).fillna(0).astype('float32')
        test[cmax] = (test[cmax] - test.AllVisits).fillna(0).astype('float32')
        test[cd] = (test[cd] ).fillna(0).astype('float32')
        test[cd7] = (test[cd7] ).fillna(0).astype('float32')
        test[cd7m] = (test[cd7m] ).fillna(0).astype('float32')
        test[cd7mm] = (test[cd7mm] ).fillna(0).astype('float32')

    for c_norm, c in zip(y_norm_cols, y_cols):
        test[c_norm] = (np.log1p(test[c]) - test.AllVisits).astype('float32')

    gc.collect()

    return test

max_periods = 16
periods = [(0,1), (1,2), (2,3), (3,4), 
           (4,5), (5,6), (6,7), (7,8),
           (0,2), (2,4),(4,6),(6,8),
           (0,4),(4,8),(8,12),(12,16),
           (0,8), (8,16), (0,12), 
           (0,16), 
          ]


site_cols = list(sites)
access_cols = list(accesses)

test, test_all = test0.copy(), test_all0.copy()

for c in y_pred_cols:
    test[c] = np.NaN
    test_all[c] = np.NaN

test1 = add_median(test, train_norm, train_norm_diff, train_norm_diff7, train_norm_diff7m, 
                   train_key, periods, max_periods, 3)

test_all1 = add_median(test_all, train_all_norm, train_all_norm_diff, train_all_norm_diff7, train_all_norm_diff7m, 
                       train_key, periods, max_periods, 5)

In [137]:
num_cols = (['median_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['mean_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['max_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['median_diff_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['median_diff7m_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['mean_diff7m_%d_%d' % (w1,w2) for (w1,w2) in periods])

import keras.backend as K

def smape_error(y_true, y_pred):
    return K.mean(K.clip(K.abs(y_pred - y_true),  0.0, 1.0), axis=-1)


def get_model(input_dim, num_sites, num_accesses, output_dim):
    
    dropout = 0.5
    regularizer = 0.00004
    main_input = Input(shape=(input_dim,), dtype='float32', name='main_input')
    site_input = Input(shape=(num_sites,), dtype='float32', name='site_input')
    access_input = Input(shape=(num_accesses,), dtype='float32', name='access_input')
    
    
    x0 = keras.layers.concatenate([main_input, site_input, access_input])
    x = Dense(200, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x0)
    x = Dropout(dropout)(x)
    x = keras.layers.concatenate([x0, x])
    x = Dense(200, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)
    x = BatchNormalization(beta_regularizer=regularizers.l2(regularizer),
                           gamma_regularizer=regularizers.l2(regularizer)
                          )(x)
    x = Dropout(dropout)(x)
    x = Dense(100, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)
    x = Dropout(dropout)(x)

    x = Dense(200, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)
    x = Dropout(dropout)(x)
    x = Dense(output_dim, activation='linear', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)

    model =  Model(inputs=[main_input, site_input, access_input], outputs=[x])
    model.compile(loss=smape_error, optimizer='adam')
    return model

group = pd.factorize(test1.Page)[0]

n_bag = 20
kf = GroupKFold(n_bag)
batch_size=4096

#print('week:', week)
test2 = test1
test_all2 = test_all1
X, Xs, Xa, y = test2[num_cols].values, test2[site_cols].values, test2[access_cols].values, test2[y_norm_cols].values
X_all, Xs_all, Xa_all, y_all = test_all2[num_cols].values, test_all2[site_cols].values, test_all2[access_cols].values, test_all2[y_norm_cols].fillna(0).values

y_true = test2[y_cols]
y_all_true = test_all2[y_cols]

models = [get_model(len(num_cols), len(site_cols), len(access_cols), len(y_cols)) for bag in range(n_bag)]

print('offset:', offset)
print('batch size:', batch_size)


best_score = 100
best_all_score = 100

save_pred = 0
saved_pred_all = 0

for n_epoch in range(10, 201, 10):
    print('************** start %d epochs **************************' % n_epoch)

    y_pred0 = np.zeros((y.shape[0], y.shape[1]))
    y_all_pred0 = np.zeros((n_bag, y_all.shape[0], y_all.shape[1]))
    for fold, (train_idx, test_idx) in enumerate(kf.split(X, y, group)):
        print('train fold', fold, end=' ')    
        model = models[fold]
        X_train, Xs_train, Xa_train, y_train = X[train_idx,:], Xs[train_idx,:], Xa[train_idx,:], y[train_idx,:]
        X_test, Xs_test, Xa_test, y_test = X[test_idx,:], Xs[test_idx,:], Xa[test_idx,:], y[test_idx,:]

        model.fit([ X_train, Xs_train, Xa_train],  y_train, 
                  epochs=10, batch_size=batch_size, verbose=0, shuffle=True, 
                  #validation_data=([X_test, Xs_test, Xa_test],  y_test)
                 )
        y_pred = model.predict([ X_test, Xs_test, Xa_test], batch_size=batch_size)
        y_all_pred = model.predict([X_all, Xs_all, Xa_all], batch_size=batch_size)

        y_pred0[test_idx,:] = y_pred
        y_all_pred0[fold,:,:]  = y_all_pred

        y_pred += test2.AllVisits.values[test_idx].reshape((-1,1))
        y_pred = np.expm1(y_pred)
        y_pred[y_pred < 0.5 * offset] = 0
        res = smape2D(test2[y_cols].values[test_idx, :], y_pred)
        y_pred = offset*((y_pred / offset).round())
        res_round = smape2D(test2[y_cols].values[test_idx, :], y_pred)

        y_all_pred += test_all2.AllVisits.values.reshape((-1,1))
        y_all_pred = np.expm1(y_all_pred)
        y_all_pred[y_all_pred < 0.5 * offset] = 0
        res_all = smape2D(test_all2[y_cols], y_all_pred)
        y_all_pred = offset*((y_all_pred / offset).round())
        res_all_round = smape2D(test_all2[y_cols], y_all_pred)
        print('smape train: %0.5f' % res, 'round: %0.5f' % res_round,
              '     smape LB: %0.5f' % res_all, 'round: %0.5f' % res_all_round)

    #y_pred0  = np.nanmedian(y_pred0, axis=0)
    y_all_pred0  = np.nanmedian(y_all_pred0, axis=0)

    y_pred0  += test2.AllVisits.values.reshape((-1,1))
    y_pred0 = np.expm1(y_pred0)
    y_pred0[y_pred0 < 0.5 * offset] = 0
    res = smape2D(y_true, y_pred0)
    print('smape train: %0.5f' % res, end=' ')
    y_pred0 = offset*((y_pred0 / offset).round())
    res_round = smape2D(y_true, y_pred0)
    print('round: %0.5f' % res_round)

    y_all_pred0 += test_all2.AllVisits.values.reshape((-1,1))
    y_all_pred0 = np.expm1(y_all_pred0)
    y_all_pred0[y_all_pred0 < 0.5 * offset] = 0
    #y_all_pred0 = y_all_pred0.round()
    res_all = smape2D(y_all_true, y_all_pred0)
    print('     smape LB: %0.5f' % res_all, end=' ')
    y_all_pred0 = offset*((y_all_pred0 / offset).round())
    res_all_round = smape2D(y_all_true, y_all_pred0)
    print('round: %0.5f' % res_all_round, end=' ')
    if res_round < best_score:
        print('saving')
        best_score = res_round
        best_all_score = res_all_round
        test.loc[:, y_pred_cols] = y_pred0
        test_all.loc[:, y_pred_cols] = y_all_pred0
    else:
        print()
    print('*************** end %d epochs **************************' % n_epoch)
print('best saved LB score:', best_all_score)

offset: 0.5
batch size: 4096
************** start 10 epochs **************************
train fold 0 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


smape train: 0.52404 round: 0.52421      smape LB: nan round: nan
train fold 1 smape train: 0.52509 round: 0.52492      smape LB: nan round: nan
train fold 2 smape train: 0.49865 round: 0.49840      smape LB: nan round: nan
train fold 3 WARNING:tensorflow:5 out of the last 1153 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7ff39c299ea0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
sma

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


smape train: 0.51245 round: 0.51198      smape LB: nan round: nan
train fold 1 smape train: 0.50835 round: 0.50845      smape LB: nan round: nan
train fold 2 smape train: 0.48052 round: 0.47977      smape LB: nan round: nan
train fold 3 smape train: 0.52292 round: 0.52241      smape LB: nan round: nan
train fold 4 smape train: 0.49954 round: 0.49892      smape LB: nan round: nan
train fold 5 smape train: 0.48427 round: 0.48386      smape LB: nan round: nan
train fold 6 smape train: 0.50721 round: 0.50696      smape LB: nan round: nan
train fold 7 smape train: 0.52002 round: 0.51991      smape LB: nan round: nan
train fold 8 smape train: 0.51297 round: 0.51268      smape LB: nan round: nan
train fold 9 smape train: 0.48094 round: 0.48036      smape LB: nan round: nan
train fold 10 smape train: 0.47980 round: 0.47939      smape LB: nan round: nan
train fold 11 smape train: 0.51047 round: 0.51018      smape LB: nan round: nan
train fold 12 smape train: 0.49174 round: 0.49115      smape LB

...

smape train: 0.46241 round: 0.46180      smape LB: nan round: nan
train fold 1 smape train: 0.46038 round: 0.45906      smape LB: nan round: nan
train fold 2 smape train: 0.43597 round: 0.43434      smape LB: nan round: nan
train fold 3 smape train: 0.46167 round: 0.45982      smape LB: nan round: nan
train fold 4 smape train: 0.45041 round: 0.44918      smape LB: nan round: nan
train fold 5 smape train: 0.44058 round: 0.43933      smape LB: nan round: nan
train fold 6 smape train: 0.44053 round: 0.43975      smape LB: nan round: nan
train fold 7 smape train: 0.46417 round: 0.46298      smape LB: nan round: nan
train fold 8 smape train: 0.47246 round: 0.47151      smape LB: nan round: nan
train fold 9 smape train: 0.43684 round: 0.43598      smape LB: nan round: nan
train fold 10 smape train: 0.43161 round: 0.43034      smape LB: nan round: nan
train fold 11 smape train: 0.45916 round: 0.45771      smape LB: nan round: nan
train fold 12 smape train: 0.43982 round: 0.43863      smape LB

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


smape train: 0.46245 round: 0.46179      smape LB: nan round: nan
train fold 1 smape train: 0.45566 round: 0.45430      smape LB: nan round: nan
train fold 2 smape train: 0.43773 round: 0.43570      smape LB: nan round: nan
train fold 3 smape train: 0.47273 round: 0.47135      smape LB: nan round: nan
train fold 4 smape train: 0.44106 round: 0.43982      smape LB: nan round: nan
train fold 5 smape train: 0.43727 round: 0.43578      smape LB: nan round: nan
train fold 6 smape train: 0.43839 round: 0.43784      smape LB: nan round: nan
train fold 7 smape train: 0.44993 round: 0.44840      smape LB: nan round: nan
train fold 8 smape train: 0.46912 round: 0.46814      smape LB: nan round: nan
train fold 9 smape train: 0.43582 round: 0.43539      smape LB: nan round: nan
train fold 10 smape train: 0.43184 round: 0.43075      smape LB: nan round: nan
train fold 11 smape train: 0.45878 round: 0.45762      smape LB: nan round: nan
train fold 12 smape train: 0.43584 round: 0.43423      smape LB

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


smape train: 0.46676 round: 0.46585      smape LB: nan round: nan
train fold 1 smape train: 0.45902 round: 0.45764      smape LB: nan round: nan
train fold 2 smape train: 0.43817 round: 0.43656      smape LB: nan round: nan
train fold 3 smape train: 0.46280 round: 0.46159      smape LB: nan round: nan
train fold 4 smape train: 0.43895 round: 0.43798      smape LB: nan round: nan
train fold 5 smape train: 0.43421 round: 0.43241      smape LB: nan round: nan
train fold 6 smape train: 0.44727 round: 0.44721      smape LB: nan round: nan
train fold 7 smape train: 0.46136 round: 0.45961      smape LB: nan round: nan
train fold 8 smape train: 0.46826 round: 0.46741      smape LB: nan round: nan
train fold 9 smape train: 0.43657 round: 0.43609      smape LB: nan round: nan
train fold 10 smape train: 0.42761 round: 0.42614      smape LB: nan round: nan
train fold 11 smape train: 0.45980 round: 0.45845      smape LB: nan round: nan
train fold 12 smape train: 0.43525 round: 0.43393      smape LB

In [138]:
filename = 'Prediction_part_1_keras_kf_12_stage2_sept_10'
path='/content/drive/My Drive/GoogleColab/Web_Traffic_Dataset/'
test_all_columns_save = [c+'_pred' for c in test_all_columns_code]
test_all_columns_save.append('Page')
test_all_save = test_all[test_all_columns_save]

test_all_save.columns = test_all_columns_date+['Page']

test_all_save.to_csv(path+'%s_test_all_save.csv' % filename, index=False)

test_all_save_columns = test_all_columns_date[:-1]+['Page']
test_all_save = test_all_save[test_all_save_columns]

test_all_save = pd.melt(test_all_save, id_vars=['Page'], var_name='Date', value_name='Visits')

#test_all_sub = test_all_id.merge(test_all_save, how='left', on=['Page','Date'])
test_all_sub = test_all_id.merge(test_all_save, on=['Page','Date'])

test_all_sub.Visits = (test_all_sub.Visits / offset).round()
#print('%.5f' % smape(test_all_sub.Visits_true, test_all_sub.Visits))

test_all_sub_sorted = test_all_sub[['Id', 'Visits']].sort_values(by='Id')

test_all_sub_sorted[['Id', 'Visits']].to_csv(path+'%s_test_sorted.csv' % filename, index=False)

#print('%.5f' % smape(test_all_sub.Visits_true, test_all_sub.Visits))
test_all_sub[['Id', 'Visits']].to_csv(path+'%s_test.csv' % filename, index=False)

test_columns_save = [c+'_pred' for c in test_columns_code]
test_columns_save.append('Page')
test_save = test[test_columns_save]
test_save.shape

test3_save_columns = [c+'_pred' for c in tests[3].columns[1:-4]][:62]
test3_save_columns.append('Page')
test_save = test_save[test3_save_columns].reset_index(drop=True)
test_save.head()


,w0_d1_pred,w0_d2_pred,w0_d3_pred,w0_d4_pred,w0_d5_pred,w0_d6_pred,w0_d0_pred,w1_d1_pred,w1_d2_pred,w1_d3_pred,w1_d4_pred,w1_d5_pred,w1_d6_pred,w1_d0_pred,w2_d1_pred,w2_d2_pred,w2_d3_pred,w2_d4_pred,w2_d5_pred,w2_d6_pred,w2_d0_pred,w3_d1_pred,w3_d2_pred,w3_d3_pred,w3_d4_pred,w3_d5_pred,w3_d6_pred,w3_d0_pred,w4_d1_pred,w4_d2_pred,w4_d3_pred,w4_d4_pred,w4_d5_pred,w4_d6_pred,w4_d0_pred,w5_d1_pred,w5_d2_pred,w5_d3_pred,w5_d4_pred,w5_d5_pred,w5_d6_pred,w5_d0_pred,w6_d1_pred,w6_d2_pred,w6_d3_pred,w6_d4_pred,w6_d5_pred,w6_d6_pred,w6_d0_pred,w7_d1_pred,w7_d2_pred,w7_d3_pred,w7_d4_pred,w7_d5_pred,w7_d6_pred,w7_d0_pred,w8_d1_pred,w8_d2_pred,w8_d3_pred,w8_d4_pred,w8_d5_pred,w8_d6_pred,Page
0,9.5,9.5,9.5,9.0,9.0,9.0,9.5,9.5,9.5,9.5,9.0,8.5,8.5,9.5,9.0,9.5,9.0,8.5,8.5,8.5,9.5,9.0,9.5,9.0,8.5,8.5,8.5,9.0,9.0,9.0,9.0,8.5,8.5,8.5,9.5,9.0,9.5,9.0,9.0,8.5,8.5,9.0,9.0,9.0,8.5,8.5,8.0,8.5,8.5,9.0,9.5,9.0,9.0,8.5,8.5,9.0,9.0,8.0,8.5,8.5,8.5,8.5,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents"
1,0.5,0.5,0.5,1.0,1.0,1.0,1.0,0.5,0.5,0.5,1.0,1.0,1.0,0.5,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,0.5,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.5,1.0,1.0,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider"
2,88.5,86.5,87.0,84.5,85.5,88.5,89.0,89.5,89.0,89.5,88.5,87.0,89.0,91.0,91.0,92.0,90.0,87.0,87.5,91.0,92.5,91.0,92.0,90.0,86.0,88.5,91.5,93.5,92.0,93.0,93.5,88.5,88.5,94.5,95.0,94.5,95.0,92.0,90.0,89.5,92.0,94.0,93.0,92.5,90.5,90.5,87.5,93.5,93.5,95.0,94.5,93.0,92.0,91.0,97.0,96.5,96.0,86.5,90.5,90.5,94.0,97.5,(1)_Ceres_de.wikipedia.org_desktop_all-agents
3,27.0,26.5,27.0,26.0,28.0,29.0,28.0,26.5,26.5,27.0,26.0,26.5,27.5,26.5,25.5,25.5,25.0,24.5,26.5,27.5,25.5,24.5,24.5,24.0,23.5,25.5,26.5,26.0,24.5,25.0,25.0,24.0,25.5,26.5,25.5,24.5,24.5,23.5,23.5,24.5,25.0,24.5,23.5,23.5,23.5,23.5,24.0,25.5,24.0,23.5,24.0,23.5,23.0,25.0,26.0,24.5,23.5,21.0,22.0,22.5,25.0,25.0,0.999…_zh.wikipedia.org_all-access_all-agents
4,36.0,35.5,34.0,30.5,29.5,33.0,36.0,34.0,34.5,34.5,31.5,29.0,30.5,33.0,31.0,31.5,30.5,28.0,27.0,28.5,31.5,29.5,30.0,28.5,26.5,25.5,27.0,29.5,28.0,28.0,28.0,25.0,25.0,27.0,29.5,28.0,28.5,27.5,25.5,24.5,26.0,28.0,26.0,26.5,25.5,24.0,22.5,24.0,25.5,25.5,25.5,25.5,24.0,23.0,25.0,26.0,26.0,23.5,24.5,23.0,23.0,24.5,1._September_de.wikipedia.org_desktop_all-agents


In [139]:
test_save.shape

(21759, 63)

In [141]:
# test_save = test_save.iloc[145063:2*145063,:].reset_index(drop=True)

test_save = test_save.iloc[7253:21759,:].reset_index(drop=True)
test3_save_columns = list(test_3_date)[:62]
test3_save_columns.append('Page')
test_save.columns = test3_save_columns
test_save.head()

test_save = pd.melt(test_save, id_vars=['Page'], var_name='Date', value_name='Visits')
test_save.Visits = (test_save.Visits / offset).round()

#print('%.5f' % smape(test_save.Visits_true, test_save.Visits))
test_save.to_csv(path+'%s_train.csv' % filename, index=False)

In [143]:
test_save

,Page,Date,Visits
0,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_all-agents",2016-09-13,19.0
1,"""Keep_me_logged_in""_extended_to_one_year_www.mediawiki.org_all-access_spider",2016-09-13,1.0
2,(1)_Ceres_de.wikipedia.org_desktop_all-agents,2016-09-13,177.0
3,0.999…_zh.wikipedia.org_all-access_all-agents,2016-09-13,54.0
4,1._September_de.wikipedia.org_desktop_all-agents,2016-09-13,72.0
...,...,...,...
899367,龍が如く6_命の詩。_ja.wikipedia.org_desktop_all-agents,2016-11-13,178.0
899368,龍千玉_zh.wikipedia.org_all-access_spider,2016-11-13,7.0
899369,龔嘉欣_zh.wikipedia.org_mobile-web_all-agents,2016-11-13,107.0
899370,龔照勝_zh.wikipedia.org_desktop_all-agents,2016-11-13,14.0
